In [1]:
pip install flaml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 206 kB 4.0 MB/s 
     |████████████████████████████████| 2.0 MB 50.2 MB/s 
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 2.2.3
    Uninstalling lightgbm-2.2.3:
      Successfully uninstalled lightgbm-2.2.3


In [2]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [3]:
import pandas as pd 
from sqlite3 import connect
import numpy as np

conn = connect('/content/drive/MyDrive/geotesouro/data/data.db')
garantia_safra = pd.read_sql("SELECT * FROM 'benef_cidadad - garantia_safra'", conn)
seguro_defeso = pd.read_sql("SELECT * FROM 'benef_cidadad - seguro_defeso'", conn)
peti = pd.read_sql("SELECT * FROM 'benef_cidadad - peti'", conn)
bpc = pd.read_sql("SELECT * FROM 'benef_cidadad - bpc'", conn)
bolsa_familia = pd.read_sql("SELECT * FROM bolsa_familia", conn)

## agg_mun
agg_municipios = pd.read_sql('SELECT * FROM agg_municipios', conn)

In [ ]:
#preds_beneficios = pd.DataFrame()
#preds_beneficios = agg_municipios[["codigo_ibge", "siafi_id", "nome"]].reset_index(drop=True)
#preds_beneficios.to_sql('preds_beneficios', con=conn, if_exists='replace', index=False)

### **Modelo - Previsão Benefícios Geral**

In [ ]:
dados_agg = pd.DataFrame(columns = ["ano_mes", "siafi_id", "cont_parcelas",	"valor_parcela_agg"])
dados_agg = pd.concat([dados_agg, garantia_safra])
dados_agg = pd.concat([dados_agg, peti])
dados_agg = pd.concat([dados_agg, bpc])
dados_agg = pd.concat([dados_agg, seguro_defeso])
dados_agg = pd.concat([dados_agg, bolsa_familia])

In [ ]:
dados_agg["siafi_id"] = dados_agg["siafi_id"].astype(float).astype(int).astype(str)
dados_agg = dados_agg.groupby(["ano_mes", "siafi_id"]).agg({"cont_parcelas":'sum',"valor_parcela_agg":'sum'}).reset_index()

dados_agg["ano"] = dados_agg["ano_mes"].astype(str).str.slice(0, 4)
dados_agg["mes"] = dados_agg["ano_mes"].astype(str).str.slice(4, 6)

dados_agg = dados_agg[["siafi_id", "ano", "valor_parcela_agg"]].groupby(["siafi_id", "ano"]).sum().reset_index()

df_melt = agg_municipios[["siafi_id", "pop_2014", "pop_2015", "pop_2016", "pop_2017", "pop_2018", "pop_2019", "pop_2020", "pop_2021"]]
pop = pd.melt(df_melt, id_vars=["siafi_id"], value_vars=["pop_2014", "pop_2015", "pop_2016", "pop_2017", "pop_2018", "pop_2019", "pop_2020", "pop_2021"])
pop["variable"] = pop["variable"].replace("pop_", "", regex = True)
pop = pop.rename(columns = {"variable": "ano", "value": "pop"})
pop["siafi_id"] = pop["siafi_id"].astype(str)

dados_agg = pd.merge(dados_agg, pop, on = ["siafi_id", "ano"], how = "left")
dados_agg = dados_agg.dropna(subset = ["pop"])
dados_agg["pop"] = dados_agg["pop"].astype(int)
dados_agg["valor_parcela_per_capta"] = dados_agg["valor_parcela_agg"] / dados_agg["pop"]
dados_agg = dados_agg.round({'valor_transf_per_capta': 2})

In [ ]:
dados_agg

,siafi_id,ano,valor_parcela_agg,pop,valor_parcela_per_capta
1,1,2014,9693260.79,46203,209.797216
2,1,2015,8973862.66,46632,192.440012
3,1,2016,7441803.88,47048,158.174713
4,1,2017,7420119.00,47451,156.374344
5,1,2018,7284288.00,45783,159.104646
...,...,...,...,...,...
50124,9997,2017,5046723.00,15016,336.089704
50125,9997,2018,5486937.00,15152,362.126254
50126,9997,2019,14498362.95,15253,950.525336
50127,9997,2020,14515892.94,15352,945.537581


In [ ]:
dados_agg = dados_agg.reset_index(drop=True)
agg_municipios["siafi_id"] = agg_municipios["siafi_id"].astype(str)
dados_agg_ = pd.merge(dados_agg[["siafi_id", "ano", "valor_parcela_per_capta"]], agg_municipios, how = "left", on = "siafi_id").rename(columns = {"valor_parcela_per_capta": "target_valor_parc"})
dados_agg_['ano'] = dados_agg_['ano'].astype(int)

### Valor Parcela Per Capta

In [ ]:
X_train = dados_agg_[(dados_agg_["ano"]!=2021)].drop(["codigo_ibge", "nome", "siafi_id", "target_valor_parc"], 1)
y_train = dados_agg_[(dados_agg_["ano"]!=2021)]["target_valor_parc"]
X_test = dados_agg_[dados_agg_["ano"]==2021].drop(["codigo_ibge", "nome", "siafi_id", "target_valor_parc"], 1)
y_test = dados_agg_[dados_agg_["ano"]==2021]["target_valor_parc"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


#### XGBoost

In [ ]:
from flaml import AutoML

automl = AutoML()
settings = {
    "time_budget": 720,  
    "metric": 'r2', 
    "estimator_list": ['xgboost'],  
    "task": 'regression'    
}
automl.fit(X_train = X_train, y_train = y_train, **settings)

[flaml.automl: 09-23 19:57:23] {2600} INFO - task = regression
INFO:flaml.automl:task = regression
[flaml.automl: 09-23 19:57:23] {2602} INFO - Data split method: uniform
INFO:flaml.automl:Data split method: uniform
[flaml.automl: 09-23 19:57:23] {2605} INFO - Evaluation method: holdout
INFO:flaml.automl:Evaluation method: holdout
[flaml.automl: 09-23 19:57:23] {2727} INFO - Minimizing error metric: 1-r2
INFO:flaml.automl:Minimizing error metric: 1-r2
[flaml.automl: 09-23 19:57:23] {2869} INFO - List of ML learners in AutoML Run: ['xgboost']
INFO:flaml.automl:List of ML learners in AutoML Run: ['xgboost']
[flaml.automl: 09-23 19:57:23] {3174} INFO - iteration 0, current learner xgboost
INFO:flaml.automl:iteration 0, current learner xgboost
[flaml.automl: 09-23 19:57:26] {3308} INFO - Estimated sufficient time budget=22682s. Estimated necessary time budget=23s.
INFO:flaml.automl:Estimated sufficient time budget=22682s. Estimated necessary time budget=23s.
[flaml.automl: 09-23 19:57:26] 

In [ ]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

Best ML leaner: xgboost
Best hyperparmeter config: {'n_estimators': 423, 'max_leaves': 1072, 'min_child_weight': 8.802381462749866, 'learning_rate': 0.06836963362165452, 'subsample': 0.827351358517848, 'colsample_bylevel': 0.46559185029557215, 'colsample_bytree': 0.5724419904724564, 'reg_alpha': 0.0027518441820229525, 'reg_lambda': 4.646649294356495}
Training duration of best run: 310.1 s


In [ ]:
automl.model.estimator

XGBRegressor(colsample_bylevel=0.46559185029557215,
             colsample_bytree=0.5724419904724564, grow_policy='lossguide',
             learning_rate=0.06836963362165452, max_depth=0, max_leaves=1072,
             min_child_weight=8.802381462749866, n_estimators=423, n_jobs=-1,
             reg_alpha=0.0027518441820229525, reg_lambda=4.646649294356495,
             subsample=0.827351358517848, tree_method='hist',
             use_label_encoder=False, verbosity=0)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

y_pred = automl.predict(X_test)

print('r2', '=', r2_score(y_test, y_pred))
print('mse', '=', mean_squared_error(y_test, y_pred))
print('rmse', '=', mean_squared_error(y_test, y_pred, squared=False))

r2 = 0.965582287197643
mse = 2900.373353788827
rmse = 53.85511446268429


In [ ]:
performance_mdl_reg = pd.DataFrame(columns = ["Tema", "Área", "Algoritmo", "R2", "MSE", "RMSE"])
performance_mdl_reg = performance_mdl_reg.append({"Tema": "Benefícios", "Área": "Geral", "Algoritmo": "XGBoost",
                                          "R2": r2_score(y_test, y_pred),
                                          "MSE": mean_squared_error(y_test, y_pred),
                                          "RMSE": mean_squared_error(y_test, y_pred, squared=False)}, ignore_index = True)

performance_mdl_reg.to_sql('performance_mdl_reg', con=conn, if_exists='append', index=False)

In [ ]:
import pickle

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_xgb_beneficios_geral.pkl', 'wb') as f:
    pickle.dump(automl, f, pickle.HIGHEST_PROTOCOL)

#### LGBM

In [ ]:
from flaml import AutoML

automl = AutoML()
settings = {
    "time_budget": 720,  
    "metric": 'r2', 
    "estimator_list": ['lgbm'],  
    "task": 'regression'    
}
automl.fit(X_train = X_train, y_train = y_train, **settings)

[flaml.automl: 09-23 20:12:08] {2600} INFO - task = regression
INFO:flaml.automl:task = regression
[flaml.automl: 09-23 20:12:08] {2602} INFO - Data split method: uniform
INFO:flaml.automl:Data split method: uniform
[flaml.automl: 09-23 20:12:08] {2605} INFO - Evaluation method: holdout
INFO:flaml.automl:Evaluation method: holdout
[flaml.automl: 09-23 20:12:08] {2727} INFO - Minimizing error metric: 1-r2
INFO:flaml.automl:Minimizing error metric: 1-r2
[flaml.automl: 09-23 20:12:08] {2869} INFO - List of ML learners in AutoML Run: ['lgbm']
INFO:flaml.automl:List of ML learners in AutoML Run: ['lgbm']
[flaml.automl: 09-23 20:12:08] {3174} INFO - iteration 0, current learner lgbm
INFO:flaml.automl:iteration 0, current learner lgbm
[flaml.automl: 09-23 20:12:09] {3308} INFO - Estimated sufficient time budget=13581s. Estimated necessary time budget=14s.
INFO:flaml.automl:Estimated sufficient time budget=13581s. Estimated necessary time budget=14s.
[flaml.automl: 09-23 20:12:09] {3360} INFO 

In [ ]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

Best ML leaner: lgbm
Best hyperparmeter config: {'n_estimators': 372, 'num_leaves': 507, 'min_child_samples': 2, 'learning_rate': 0.14357025802388537, 'log_max_bin': 10, 'colsample_bytree': 0.517801258265659, 'reg_alpha': 0.011572343074847936, 'reg_lambda': 0.006047099428180339}
Training duration of best run: 276.1 s


In [ ]:
automl.model.estimator

LGBMRegressor(colsample_bytree=0.517801258265659,
              learning_rate=0.14357025802388537, max_bin=1023,
              min_child_samples=2, n_estimators=372, num_leaves=507,
              reg_alpha=0.011572343074847936, reg_lambda=0.006047099428180339,
              verbose=-1)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

y_pred = automl.predict(X_test)

print('r2', '=', r2_score(y_test, y_pred))
print('mse', '=', mean_squared_error(y_test, y_pred))
print('rmse', '=', mean_squared_error(y_test, y_pred, squared=False))

r2 = 0.9635285262528974
mse = 3073.443352757052
rmse = 55.438644939762476


In [ ]:
performance_mdl_reg = pd.DataFrame(columns = ["Tema", "Área", "Algoritmo", "R2", "MSE", "RMSE"])
performance_mdl_reg = performance_mdl_reg.append({"Tema": "Benefícios", "Área": "Geral", "Algoritmo": "LGBM",
                                          "R2": r2_score(y_test, y_pred),
                                          "MSE": mean_squared_error(y_test, y_pred),
                                          "RMSE": mean_squared_error(y_test, y_pred, squared=False)}, ignore_index = True)

performance_mdl_reg.to_sql('performance_mdl_reg', con=conn, if_exists='append', index=False)

In [ ]:
import pickle

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_lgbm_beneficios_geral.pkl', 'wb') as f:
    pickle.dump(automl, f, pickle.HIGHEST_PROTOCOL)

#### CatBoost

In [ ]:
from flaml import AutoML

automl = AutoML()
settings = {
    "time_budget": 720,  
    "metric": 'r2', 
    "estimator_list": ['catboost'],  
    "task": 'regression'    
}
automl.fit(X_train = X_train, y_train = y_train, **settings)

[flaml.automl: 09-23 20:27:06] {2600} INFO - task = regression
INFO:flaml.automl:task = regression
[flaml.automl: 09-23 20:27:06] {2602} INFO - Data split method: uniform
INFO:flaml.automl:Data split method: uniform
[flaml.automl: 09-23 20:27:06] {2605} INFO - Evaluation method: holdout
INFO:flaml.automl:Evaluation method: holdout
[flaml.automl: 09-23 20:27:06] {2727} INFO - Minimizing error metric: 1-r2
INFO:flaml.automl:Minimizing error metric: 1-r2
[flaml.automl: 09-23 20:27:06] {2869} INFO - List of ML learners in AutoML Run: ['catboost']
INFO:flaml.automl:List of ML learners in AutoML Run: ['catboost']
[flaml.automl: 09-23 20:27:06] {3174} INFO - iteration 0, current learner catboost
INFO:flaml.automl:iteration 0, current learner catboost
[flaml.automl: 09-23 20:29:41] {3308} INFO - Estimated sufficient time budget=1550765s. Estimated necessary time budget=1551s.
INFO:flaml.automl:Estimated sufficient time budget=1550765s. Estimated necessary time budget=1551s.
[flaml.automl: 09-2

In [ ]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

Best ML leaner: catboost
Best hyperparmeter config: {'early_stopping_rounds': 10, 'learning_rate': 0.06233639237958607, 'n_estimators': 3939}
Training duration of best run: 371.3 s


In [ ]:
automl.model.estimator

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

y_pred = automl.predict(X_test)

print('r2', '=', r2_score(y_test, y_pred))
print('mse', '=', mean_squared_error(y_test, y_pred))
print('rmse', '=', mean_squared_error(y_test, y_pred, squared=False))

r2 = 0.9426119705695425
mse = 4836.077061319132
rmse = 69.54190866893957


In [ ]:
performance_mdl_reg = pd.DataFrame(columns = ["Tema", "Área", "Algoritmo", "R2", "MSE", "RMSE"])
performance_mdl_reg = performance_mdl_reg.append({"Tema": "Benefícios", "Área": "Geral", "Algoritmo": "CatBoost",
                                          "R2": r2_score(y_test, y_pred),
                                          "MSE": mean_squared_error(y_test, y_pred),
                                          "RMSE": mean_squared_error(y_test, y_pred, squared=False)}, ignore_index = True)

performance_mdl_reg.to_sql('performance_mdl_reg', con=conn, if_exists='append', index=False)

In [ ]:
import pickle

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_cb_beneficios_geral.pkl', 'wb') as f:
    pickle.dump(automl, f, pickle.HIGHEST_PROTOCOL)

#### Ensemble

In [ ]:
from sklearn.linear_model import LinearRegression
from flaml.ml import sklearn_metric_loss_score

pred_train_ensem = pd.DataFrame()
pred_test_ensem = pd.DataFrame()

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_xgb_beneficios_geral.pkl', 'rb') as f:
    mdl_xgb = pickle.load(f)

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_lgbm_beneficios_geral.pkl', 'rb') as f:
    mdl_lgbm = pickle.load(f)

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_cb_beneficios_geral.pkl', 'rb') as f:
    mdl_cb = pickle.load(f)

pred_train_ensem["xgb_pred_geral_valor_per_capta"] = mdl_xgb.predict(X_train)
pred_test_ensem["xgb_pred_geral_valor_per_capta"] = mdl_xgb.predict(X_test)
pred_train_ensem["lgbm_pred_geral_valor_per_capta"] = mdl_lgbm.predict(X_train)
pred_test_ensem["lgbm_pred_geral_valor_per_capta"] = mdl_lgbm.predict(X_test)
pred_train_ensem["cb_pred_geral_valor_per_capta"] = mdl_cb.predict(X_train)
pred_test_ensem["cb_pred_geral_valor_per_capta"] = mdl_cb.predict(X_test)

pred_train_ensem["y_train"] = y_train.reset_index(drop=True)
pred_test_ensem["y_test"] = y_test.reset_index(drop=True)

X_train_ensem = pred_train_ensem.drop(["y_train"], 1)
X_test_ensem = pred_test_ensem.drop(["y_test"], 1)
y_train_ensem = pred_train_ensem["y_train"]
y_test_ensem = pred_test_ensem["y_test"]

clf = LinearRegression().fit(X_train_ensem, y_train_ensem)
y_pred = clf.predict(X_test_ensem)
print('r2', '=', r2_score(y_test_ensem, y_pred))
print('mse', '=', mean_squared_error(y_test_ensem, y_pred))
print('rmse', '=', mean_squared_error(y_test_ensem, y_pred, squared=False))


r2 = 0.9635155282140453
mse = 3074.5386947326992
rmse = 55.44852292651896


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


In [ ]:
performance_mdl_reg = pd.DataFrame(columns = ["Tema", "Área", "Algoritmo", "R2", "MSE", "RMSE"])
performance_mdl_reg = performance_mdl_reg.append({"Tema": "Benefícios", "Área": "Geral", "Algoritmo": "Ensemble",
                                          "R2": r2_score(y_test_ensem, y_pred),
                                          "MSE": mean_squared_error(y_test_ensem, y_pred),
                                          "RMSE": mean_squared_error(y_test_ensem, y_pred, squared=False)}, ignore_index = True)

performance_mdl_reg.to_sql('performance_mdl_reg', con=conn, if_exists='append', index=False)

In [ ]:
import pickle

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_ensem_beneficios_geral.pkl', 'wb') as f:
    pickle.dump(clf, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
X_preds = agg_municipios.drop(["codigo_ibge", "nome", "siafi_id"], 1)
X_preds["ano"] = 2022

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [ ]:
preds = pd.DataFrame()
preds["xgb_pred_geral_valor_per_capta"] = mdl_xgb.predict(X_preds)
preds["lgbm_pred_geral_valor_per_capta"] = mdl_lgbm.predict(X_preds)
preds["cb_pred_geral_valor_per_capta"] = mdl_cb.predict(X_preds)

preds["candido_pred_geral_valor_per_capta"] = clf.predict(preds)

In [ ]:
preds_beneficios = agg_municipios[["codigo_ibge", "siafi_id", "nome"]]
preds_beneficios = pd.concat([preds_beneficios, preds], axis = 1)
preds_beneficios

,codigo_ibge,siafi_id,nome,xgb_pred_geral_valor_per_capta,lgbm_pred_geral_valor_per_capta,cb_pred_geral_valor_per_capta,candido_pred_geral_valor_per_capta
0,5200050,1050,Abadia de Goiás,349.573975,355.753701,314.863405,357.951957
1,3100104,4001,Abadia dos Dourados,294.366028,292.191176,340.630935,289.443037
2,5200100,9201,Abadiânia,222.412933,222.013170,246.622988,220.653143
3,3100203,4003,Abaeté,598.802124,597.373253,553.514209,599.758227
4,1500107,401,Abaetetuba,1129.879272,1122.874233,1116.076801,1122.887620
...,...,...,...,...,...,...,...
5565,2933604,3971,Xique-Xique,1428.087891,1446.442192,1362.260428,1450.142965
5566,2517407,542,Zabelê,527.753845,523.915775,523.662721,523.875957
5567,3557154,2973,Zacarias,115.196381,115.025079,158.367872,112.644512
5568,2114007,1287,Zé Doca,852.595154,847.160698,847.909986,846.883978


In [ ]:
preds_beneficios.to_sql('preds_beneficios', con=conn, if_exists='replace', index=False)

### **Modelos Residuais - Previsão Benefícios**

### Garantia-Safra

#### Binário

In [4]:
pred_train_ensem = pd.DataFrame()
pred_test_ensem = pd.DataFrame()

In [5]:
garantia_safra_ = garantia_safra[garantia_safra["valor_parcela_agg"]!=0]
garantia_safra_ = garantia_safra_.reset_index(drop=True)
garantia_safra_["ano"] = garantia_safra_["ano_mes"].astype(str).str.slice(0, 4)
garantia_safra_["mes"] = garantia_safra_["ano_mes"].astype(str).str.slice(4, 6)
garantia_safra_ = garantia_safra_[["siafi_id", "ano", "valor_parcela_agg"]].groupby(["siafi_id", "ano"]).sum().reset_index()

In [6]:
list_cols = agg_municipios.columns.to_list()
list_cols.append("target")
list_cols.append("ano")
dataset = pd.DataFrame(columns = list_cols)

for a in garantia_safra_["ano"].unique():
  target = garantia_safra_[garantia_safra_["ano"]==a]
  target["target"] = 1
  target = pd.merge(agg_municipios, target[["siafi_id", "target"]], how = "left", on = "siafi_id")
  target["target"] = target["target"].fillna(0).astype(int)
  target["ano"] = a
  dataset = dataset.append(target, ignore_index=True)

dataset["ano"] = dataset["ano"].astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
X_train = dataset[dataset["ano"]!=2021].drop(["codigo_ibge", "nome", "siafi_id", "target"], 1)
y_train = dataset[dataset["ano"]!=2021]["target"]
X_test = dataset[dataset["ano"]==2021].drop(["codigo_ibge", "nome", "siafi_id", "target"], 1)
y_test = dataset[dataset["ano"]==2021]["target"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


##### XGBoost

In [8]:
from flaml import AutoML

automl = AutoML()
settings = {
    "time_budget": 360,  
    "metric": 'roc_auc', 
    "estimator_list": ['xgboost'],  
    "task": 'classification' 
}

automl.fit(X_train = X_train, y_train = y_train, **settings)

[flaml.automl: 09-24 09:17:54] {2600} INFO - task = classification
INFO:flaml.automl:task = classification
[flaml.automl: 09-24 09:17:54] {2602} INFO - Data split method: stratified
INFO:flaml.automl:Data split method: stratified
[flaml.automl: 09-24 09:17:54] {2605} INFO - Evaluation method: holdout
INFO:flaml.automl:Evaluation method: holdout
[flaml.automl: 09-24 09:17:55] {2727} INFO - Minimizing error metric: 1-roc_auc
INFO:flaml.automl:Minimizing error metric: 1-roc_auc
[flaml.automl: 09-24 09:17:55] {2869} INFO - List of ML learners in AutoML Run: ['xgboost']
INFO:flaml.automl:List of ML learners in AutoML Run: ['xgboost']
[flaml.automl: 09-24 09:17:55] {3174} INFO - iteration 0, current learner xgboost
INFO:flaml.automl:iteration 0, current learner xgboost
[flaml.automl: 09-24 09:17:59] {3308} INFO - Estimated sufficient time budget=148071s. Estimated necessary time budget=148s.
INFO:flaml.automl:Estimated sufficient time budget=148071s. Estimated necessary time budget=148s.
[fl

In [9]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best accuracy on validation data: {0:.4g}'.format(1-automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

Best ML leaner: xgboost
Best hyperparmeter config: {'n_estimators': 119, 'max_leaves': 1539, 'min_child_weight': 0.11870233476745663, 'learning_rate': 0.06748914790252888, 'subsample': 0.827351358517848, 'colsample_bylevel': 0.5127241764310984, 'colsample_bytree': 0.8212321291437105, 'reg_alpha': 0.0036351082935974586, 'reg_lambda': 1.9488000465238167, 'FLAML_sample_size': 40104}
Best accuracy on validation data: 0.9951
Training duration of best run: 87.3 s


In [10]:
automl.model.estimator

XGBClassifier(colsample_bylevel=0.5127241764310984,
              colsample_bytree=0.8212321291437105, grow_policy='lossguide',
              learning_rate=0.06748914790252888, max_depth=0, max_leaves=1539,
              min_child_weight=0.11870233476745663, n_estimators=119, n_jobs=-1,
              reg_alpha=0.0036351082935974586, reg_lambda=1.9488000465238167,
              subsample=0.827351358517848, tree_method='hist',
              use_label_encoder=False, verbosity=0)

In [11]:
from flaml.ml import sklearn_metric_loss_score

y_pred = automl.predict(X_test)
y_pred_proba = automl.predict_proba(X_test)[:,1]
print('accuracy', '=', 1 - sklearn_metric_loss_score('accuracy', y_pred.astype(int), y_test.astype(int)))
print('roc_auc', '=', 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test.astype(int)))
print('log_loss', '=', sklearn_metric_loss_score('log_loss', y_pred_proba, y_test.astype(int)))

accuracy = 0.9317773788150808
roc_auc = 0.979581643963749
log_loss = 0.23209348598518012


In [ ]:
performance_mdl = pd.DataFrame(columns = ["Tema", "Área", "Algoritmo", "Accuracy", "ROC_AUC", "Log_Loss"])
performance_mdl = performance_mdl.append({"Tema": "Benefícios", "Área": "Residual - Garantia-Safra - Binário", "Algoritmo": "XGBoost",
                                          "Accuracy": 1 - sklearn_metric_loss_score('accuracy', y_pred.astype(int), y_test.astype(int)),
                                          "ROC_AUC": 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test.astype(int)),
                                          "Log_Loss": sklearn_metric_loss_score('log_loss', y_pred_proba, y_test.astype(int))}, ignore_index = True)

performance_mdl.to_sql('performance_mdl', con=conn, if_exists='append', index=False)

In [12]:
import pickle

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_xgb_beneficios_resid_garantia_safra_binario.pkl', 'wb') as f:
    pickle.dump(automl, f, pickle.HIGHEST_PROTOCOL)

##### LGBM

In [13]:
from flaml import AutoML

automl = AutoML()
settings = {
    "time_budget": 360,  
    "metric": 'roc_auc', 
    "estimator_list": ['lgbm'],  
    "task": 'classification' 
}

automl.fit(X_train = X_train, y_train = y_train, **settings)

[flaml.automl: 09-24 09:24:27] {2600} INFO - task = classification
INFO:flaml.automl:task = classification
[flaml.automl: 09-24 09:24:27] {2602} INFO - Data split method: stratified
INFO:flaml.automl:Data split method: stratified
[flaml.automl: 09-24 09:24:27] {2605} INFO - Evaluation method: holdout
INFO:flaml.automl:Evaluation method: holdout
[flaml.automl: 09-24 09:24:27] {2727} INFO - Minimizing error metric: 1-roc_auc
INFO:flaml.automl:Minimizing error metric: 1-roc_auc
[flaml.automl: 09-24 09:24:27] {2869} INFO - List of ML learners in AutoML Run: ['lgbm']
INFO:flaml.automl:List of ML learners in AutoML Run: ['lgbm']
[flaml.automl: 09-24 09:24:27] {3174} INFO - iteration 0, current learner lgbm
INFO:flaml.automl:iteration 0, current learner lgbm
[flaml.automl: 09-24 09:24:28] {3308} INFO - Estimated sufficient time budget=28386s. Estimated necessary time budget=28s.
INFO:flaml.automl:Estimated sufficient time budget=28386s. Estimated necessary time budget=28s.
[flaml.automl: 09-2

In [14]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best accuracy on validation data: {0:.4g}'.format(1-automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

Best ML leaner: lgbm
Best hyperparmeter config: {'n_estimators': 279, 'num_leaves': 276, 'min_child_samples': 2, 'learning_rate': 0.32069904554817585, 'log_max_bin': 10, 'colsample_bytree': 0.492219639722459, 'reg_alpha': 0.0020145913289276885, 'reg_lambda': 0.003541197549809508, 'FLAML_sample_size': 40104}
Best accuracy on validation data: 0.995
Training duration of best run: 44.2 s


In [15]:
automl.model.estimator

LGBMClassifier(colsample_bytree=0.492219639722459,
               learning_rate=0.32069904554817585, max_bin=1023,
               min_child_samples=2, n_estimators=279, num_leaves=276,
               reg_alpha=0.0020145913289276885, reg_lambda=0.003541197549809508,
               verbose=-1)

In [16]:
from flaml.ml import sklearn_metric_loss_score

y_pred = automl.predict(X_test)
y_pred_proba = automl.predict_proba(X_test)[:,1]
print('accuracy', '=', 1 - sklearn_metric_loss_score('accuracy', y_pred.astype(int), y_test.astype(int)))
print('roc_auc', '=', 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test.astype(int)))
print('log_loss', '=', sklearn_metric_loss_score('log_loss', y_pred_proba, y_test.astype(int)))

accuracy = 0.9315978456014362
roc_auc = 0.9778812267308903
log_loss = 0.7367856796314254


In [ ]:
performance_mdl = pd.DataFrame(columns = ["Tema", "Área", "Algoritmo", "Accuracy", "ROC_AUC", "Log_Loss"])
performance_mdl = performance_mdl.append({"Tema": "Benefícios", "Área": "Residual - Garantia-Safra - Binário", "Algoritmo": "LGBM",
                                          "Accuracy": 1 - sklearn_metric_loss_score('accuracy', y_pred.astype(int), y_test.astype(int)),
                                          "ROC_AUC": 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test.astype(int)),
                                          "Log_Loss": sklearn_metric_loss_score('log_loss', y_pred_proba, y_test.astype(int))}, ignore_index = True)

performance_mdl.to_sql('performance_mdl', con=conn, if_exists='append', index=False)

In [17]:
import pickle

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_lgbm_beneficios_resid_garantia_safra_binario.pkl', 'wb') as f:
    pickle.dump(automl, f, pickle.HIGHEST_PROTOCOL)

##### CatBoost


In [18]:
from flaml import AutoML

automl = AutoML()
settings = {
    "time_budget": 360,  
    "metric": 'roc_auc', 
    "estimator_list": ['catboost'],  
    "task": 'classification' 
}

automl.fit(X_train = X_train, y_train = y_train, **settings)

[flaml.automl: 09-24 09:31:02] {2600} INFO - task = classification
INFO:flaml.automl:task = classification
[flaml.automl: 09-24 09:31:02] {2602} INFO - Data split method: stratified
INFO:flaml.automl:Data split method: stratified
[flaml.automl: 09-24 09:31:02] {2605} INFO - Evaluation method: holdout
INFO:flaml.automl:Evaluation method: holdout
[flaml.automl: 09-24 09:31:02] {2727} INFO - Minimizing error metric: 1-roc_auc
INFO:flaml.automl:Minimizing error metric: 1-roc_auc
[flaml.automl: 09-24 09:31:02] {2869} INFO - List of ML learners in AutoML Run: ['catboost']
INFO:flaml.automl:List of ML learners in AutoML Run: ['catboost']
[flaml.automl: 09-24 09:31:02] {3174} INFO - iteration 0, current learner catboost
INFO:flaml.automl:iteration 0, current learner catboost
[flaml.automl: 09-24 09:31:32] {3308} INFO - Estimated sufficient time budget=1182591s. Estimated necessary time budget=1183s.
INFO:flaml.automl:Estimated sufficient time budget=1182591s. Estimated necessary time budget=11

In [19]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best accuracy on validation data: {0:.4g}'.format(1-automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

Best ML leaner: catboost
Best hyperparmeter config: {'early_stopping_rounds': 10, 'learning_rate': 0.06233639237958607, 'n_estimators': 133, 'FLAML_sample_size': 40104}
Best accuracy on validation data: 0.992
Training duration of best run: 45.11 s


In [20]:
automl.model.estimator

In [21]:
from flaml.ml import sklearn_metric_loss_score

y_pred = automl.predict(X_test)
y_pred_proba = automl.predict_proba(X_test)[:,1]
print('accuracy', '=', 1 - sklearn_metric_loss_score('accuracy', y_pred.astype(int), y_test.astype(int)))
print('roc_auc', '=', 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test.astype(int)))
print('log_loss', '=', sklearn_metric_loss_score('log_loss', y_pred_proba, y_test.astype(int)))

accuracy = 0.9254937163375224
roc_auc = 0.9829988197428844
log_loss = 0.1542939752639196


In [ ]:
performance_mdl = pd.DataFrame(columns = ["Tema", "Área", "Algoritmo", "Accuracy", "ROC_AUC", "Log_Loss"])
performance_mdl = performance_mdl.append({"Tema": "Benefícios", "Área": "Residual - Garantia-Safra - Binário", "Algoritmo": "CatBoost",
                                          "Accuracy": 1 - sklearn_metric_loss_score('accuracy', y_pred.astype(int), y_test.astype(int)),
                                          "ROC_AUC": 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test.astype(int)),
                                          "Log_Loss": sklearn_metric_loss_score('log_loss', y_pred_proba, y_test.astype(int))}, ignore_index = True)

performance_mdl.to_sql('performance_mdl', con=conn, if_exists='append', index=False)

In [22]:
import pickle

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_cb_beneficios_resid_garantia_safra_binario.pkl', 'wb') as f:
    pickle.dump(automl, f, pickle.HIGHEST_PROTOCOL)

##### Ensemble

In [25]:
from sklearn.linear_model import LogisticRegression
from flaml.ml import sklearn_metric_loss_score
import pickle

pred_train_ensem = pd.DataFrame()
pred_test_ensem = pd.DataFrame()

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_xgb_beneficios_resid_garantia_safra_binario.pkl', 'rb') as f:
    mdl_xgb = pickle.load(f)

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_lgbm_beneficios_resid_garantia_safra_binario.pkl', 'rb') as f:
    mdl_lgbm = pickle.load(f)

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_cb_beneficios_resid_garantia_safra_binario.pkl', 'rb') as f:
    mdl_cb = pickle.load(f)

pred_train_ensem[["xgb_proba_0_resid_garantia_safra_binario", "xgb_proba_1_resid_garantia_safra_binario"]] = mdl_xgb.predict_proba(X_train)
pred_test_ensem[["xgb_proba_0_resid_garantia_safra_binario", "xgb_proba_1_resid_garantia_safra_binario"]] = mdl_xgb.predict_proba(X_test)
pred_train_ensem[["lgbm_proba_0_resid_garantia_safra_binario", "lgbm_proba_1_resid_garantia_safra_binario"]] = mdl_lgbm.predict_proba(X_train)
pred_test_ensem[["lgbm_proba_0_resid_garantia_safra_binario", "lgbm_proba_1_resid_garantia_safra_binario"]] = mdl_lgbm.predict_proba(X_test)
pred_train_ensem[["cb_proba_0_resid_garantia_safra_binario", "cb_proba_1_resid_garantia_safra_binario"]] = mdl_cb.predict_proba(X_train)
pred_test_ensem[["cb_proba_0_resid_garantia_safra_binario", "cb_proba_1_resid_garantia_safra_binario"]] = mdl_cb.predict_proba(X_test)

pred_train_ensem["y_train"] = y_train.reset_index(drop=True)
pred_test_ensem["y_test"] = y_test.reset_index(drop=True)

X_train_ensem = pred_train_ensem.drop(["y_train"], 1)
X_test_ensem = pred_test_ensem.drop(["y_test"], 1)
y_train_ensem = pred_train_ensem["y_train"]
y_test_ensem = pred_test_ensem["y_test"]

clf = LogisticRegression().fit(X_train_ensem, y_train_ensem.astype(int))
y_pred = clf.predict(X_test_ensem)
y_pred_proba = clf.predict_proba(X_test_ensem)[:,1]
print('accuracy', '=', 1 - sklearn_metric_loss_score('accuracy', y_pred.astype(int), y_test.astype(int)))
print('roc_auc', '=', 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test.astype(int)))
print('log_loss', '=', sklearn_metric_loss_score('log_loss', y_pred_proba, y_test.astype(int)))

performance_mdl = pd.DataFrame(columns = ["Tema", "Área", "Algoritmo", "Accuracy", "ROC_AUC", "Log_Loss"])
performance_mdl = performance_mdl.append({"Tema": "Benefícios", "Área": "Residual - Garantia-Safra - Binário", "Algoritmo": "Ensemble",
                                          "Accuracy": 1 - sklearn_metric_loss_score('accuracy', y_pred.astype(int), y_test.astype(int)),
                                          "ROC_AUC": 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test.astype(int)),
                                          "Log_Loss": sklearn_metric_loss_score('log_loss', y_pred_proba, y_test.astype(int))}, ignore_index = True)

performance_mdl.to_sql('performance_mdl', con=conn, if_exists='append', index=False)

preds_beneficios = pd.DataFrame()

X_preds = agg_municipios.drop(["codigo_ibge", "nome", "siafi_id"], 1)
X_preds["ano"] = 2022

preds_beneficios[["xgb_proba_0_resid_garantia_safra_binario", "xgb_proba_1_resid_garantia_safra_binario"]] = mdl_xgb.predict_proba(X_preds)
preds_beneficios[["lgbm_proba_0_resid_garantia_safra_binario", "lgbm_proba_1_resid_garantia_safra_binario"]] = mdl_lgbm.predict_proba(X_preds)
preds_beneficios[["cb_proba_0_resid_garantia_safra_binario", "cb_proba_1_resid_garantia_safra_binario"]] = mdl_cb.predict_proba(X_preds)

preds_beneficios[["candido_pred_proba_0_resid_garantia_safra_binario", "candido_pred_proba_1_resid_garantia_safra_binario"]] = clf.predict_proba(preds_beneficios)
preds_beneficios["candido_pred_garantia_safra_binario"] = clf.predict(preds_beneficios.drop(["candido_pred_proba_0_resid_garantia_safra_binario", "candido_pred_proba_1_resid_garantia_safra_binario"], 1))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


accuracy = 0.9315978456014362
roc_auc = 0.9837848139023608
log_loss = 0.5657260002952976


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


In [27]:
preds_beneficios_db = pd.read_sql('select * from preds_beneficios', con = conn)
preds_beneficios_db = pd.concat([preds_beneficios_db, preds_beneficios], axis = 1)
preds_beneficios_db.to_sql('preds_beneficios', con=conn, if_exists='replace', index=False)

### Seguro Defeso

#### Binário

In [5]:
seguro_defeso_ = seguro_defeso[seguro_defeso["valor_parcela_agg"]!=0]
seguro_defeso_["siafi_id"] = seguro_defeso_["siafi_id"].str.replace(".0", "", regex = True).astype(str)
seguro_defeso_ = seguro_defeso_.reset_index(drop=True)
seguro_defeso_["ano"] = seguro_defeso_["ano_mes"].astype(str).str.slice(0, 4)
seguro_defeso_["mes"] = seguro_defeso_["ano_mes"].astype(str).str.slice(4, 6)
seguro_defeso_ = seguro_defeso_[["siafi_id", "ano", "valor_parcela_agg"]].groupby(["siafi_id", "ano"]).sum().reset_index()
seguro_defeso_ = seguro_defeso_[seguro_defeso_["siafi_id"]!=""][seguro_defeso_["siafi_id"]!="0"].astype(int)
#seguro_defeso_["siafi_id"] = seguro_defeso_["siafi_id"].astype(str)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


In [6]:
list_cols = agg_municipios.columns.to_list()
list_cols.append("target")
list_cols.append("ano")
dataset = pd.DataFrame(columns = list_cols)


for a in seguro_defeso_["ano"].unique():
  target = seguro_defeso_[seguro_defeso_["ano"]==a]
  target["target"] = 1
  target = pd.merge(agg_municipios, target[["siafi_id", "target"]], how = "left", on = "siafi_id")
  target["target"] = target["target"].fillna(0).astype(int)
  target["ano"] = a
  dataset = dataset.append(target, ignore_index=True)

dataset["ano"] = dataset["ano"].astype(int)
dataset = dataset.drop_duplicates()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [7]:
X_train = dataset[dataset["ano"]!=2021].drop(["codigo_ibge", "nome", "siafi_id", "target"], 1)
y_train = dataset[dataset["ano"]!=2021]["target"]
X_test = dataset[dataset["ano"]==2021].drop(["codigo_ibge", "nome", "siafi_id", "target"], 1)
y_test = dataset[dataset["ano"]==2021]["target"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


##### XGBoost

In [62]:
from flaml import AutoML

automl = AutoML()
settings = {
    "time_budget": 360,  
    "metric": 'roc_auc', 
    "estimator_list": ['xgboost'],  
    "task": 'classification' 
}

automl.fit(X_train = X_train, y_train = y_train, **settings)

[flaml.automl: 09-24 11:01:13] {2600} INFO - task = classification
INFO:flaml.automl:task = classification
[flaml.automl: 09-24 11:01:14] {2602} INFO - Data split method: stratified
INFO:flaml.automl:Data split method: stratified
[flaml.automl: 09-24 11:01:14] {2605} INFO - Evaluation method: holdout
INFO:flaml.automl:Evaluation method: holdout
[flaml.automl: 09-24 11:01:14] {2727} INFO - Minimizing error metric: 1-roc_auc
INFO:flaml.automl:Minimizing error metric: 1-roc_auc
[flaml.automl: 09-24 11:01:14] {2869} INFO - List of ML learners in AutoML Run: ['xgboost']
INFO:flaml.automl:List of ML learners in AutoML Run: ['xgboost']
[flaml.automl: 09-24 11:01:14] {3174} INFO - iteration 0, current learner xgboost
INFO:flaml.automl:iteration 0, current learner xgboost
[flaml.automl: 09-24 11:01:15] {3308} INFO - Estimated sufficient time budget=42071s. Estimated necessary time budget=42s.
INFO:flaml.automl:Estimated sufficient time budget=42071s. Estimated necessary time budget=42s.
[flaml.

In [63]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best accuracy on validation data: {0:.4g}'.format(1-automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

Best ML leaner: xgboost
Best hyperparmeter config: {'n_estimators': 93, 'max_leaves': 954, 'min_child_weight': 0.363003367868085, 'learning_rate': 0.0050255399202444, 'subsample': 0.7729863928409147, 'colsample_bylevel': 0.4457341468875404, 'colsample_bytree': 0.6305989052610687, 'reg_alpha': 0.2456919760720914, 'reg_lambda': 0.13085282563003992, 'FLAML_sample_size': 40104}
Best accuracy on validation data: 0.9818
Training duration of best run: 71.83 s


In [64]:
automl.model.estimator

XGBClassifier(colsample_bylevel=0.4457341468875404,
              colsample_bytree=0.6305989052610687, grow_policy='lossguide',
              learning_rate=0.0050255399202444, max_depth=0, max_leaves=954,
              min_child_weight=0.363003367868085, n_estimators=93, n_jobs=-1,
              reg_alpha=0.2456919760720914, reg_lambda=0.13085282563003992,
              subsample=0.7729863928409147, tree_method='hist',
              use_label_encoder=False, verbosity=0)

In [65]:
from flaml.ml import sklearn_metric_loss_score

y_pred = automl.predict(X_test)
y_pred_proba = automl.predict_proba(X_test)[:,1]
print('accuracy', '=', 1 - sklearn_metric_loss_score('accuracy', y_pred.astype(int), y_test.astype(int)))
print('roc_auc', '=', 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test.astype(int)))
print('log_loss', '=', sklearn_metric_loss_score('log_loss', y_pred_proba, y_test.astype(int)))

accuracy = 0.9509874326750449
roc_auc = 0.981667437047731
log_loss = 0.43578594901617385


In [ ]:
performance_mdl = pd.DataFrame(columns = ["Tema", "Área", "Algoritmo", "Accuracy", "ROC_AUC", "Log_Loss"])
performance_mdl = performance_mdl.append({"Tema": "Benefícios", "Área": "Residual - Seguro Defeso - Binário", "Algoritmo": "XGBoost",
                                          "Accuracy": 1 - sklearn_metric_loss_score('accuracy', y_pred.astype(int), y_test.astype(int)),
                                          "ROC_AUC": 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test.astype(int)),
                                          "Log_Loss": sklearn_metric_loss_score('log_loss', y_pred_proba, y_test.astype(int))}, ignore_index = True)

performance_mdl.to_sql('performance_mdl', con=conn, if_exists='append', index=False)

In [66]:
import pickle

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_xgb_beneficios_resid_seguro_defeso_binario.pkl', 'wb') as f:
    pickle.dump(automl, f, pickle.HIGHEST_PROTOCOL)

##### LGBM

In [17]:
from flaml import AutoML

automl = AutoML()
settings = {
    "time_budget": 250,  
    "metric": 'roc_auc', 
    "estimator_list": ['lgbm'],  
    "task": 'classification' 
}

automl.fit(X_train = X_train, y_train = y_train, **settings)

[flaml.automl: 09-24 13:55:31] {2600} INFO - task = classification
INFO:flaml.automl:task = classification
[flaml.automl: 09-24 13:55:31] {2602} INFO - Data split method: stratified
INFO:flaml.automl:Data split method: stratified
[flaml.automl: 09-24 13:55:31] {2605} INFO - Evaluation method: holdout
INFO:flaml.automl:Evaluation method: holdout
[flaml.automl: 09-24 13:55:31] {2727} INFO - Minimizing error metric: 1-roc_auc
INFO:flaml.automl:Minimizing error metric: 1-roc_auc
[flaml.automl: 09-24 13:55:31] {2869} INFO - List of ML learners in AutoML Run: ['lgbm']
INFO:flaml.automl:List of ML learners in AutoML Run: ['lgbm']
[flaml.automl: 09-24 13:55:31] {3174} INFO - iteration 0, current learner lgbm
INFO:flaml.automl:iteration 0, current learner lgbm
[flaml.automl: 09-24 13:55:32] {3308} INFO - Estimated sufficient time budget=31795s. Estimated necessary time budget=32s.
INFO:flaml.automl:Estimated sufficient time budget=31795s. Estimated necessary time budget=32s.
[flaml.automl: 09-2

In [18]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best accuracy on validation data: {0:.4g}'.format(1-automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

Best ML leaner: lgbm
Best hyperparmeter config: {'n_estimators': 645, 'num_leaves': 120, 'min_child_samples': 2, 'learning_rate': 0.1632300183443432, 'log_max_bin': 8, 'colsample_bytree': 0.43638522659806045, 'reg_alpha': 0.02046640007359354, 'reg_lambda': 0.0032197962023686427, 'FLAML_sample_size': 40104}
Best accuracy on validation data: 0.99
Training duration of best run: 66.18 s


In [19]:
automl.model.estimator

LGBMClassifier(colsample_bytree=0.43638522659806045,
               learning_rate=0.1632300183443432, max_bin=255,
               min_child_samples=2, n_estimators=645, num_leaves=120,
               reg_alpha=0.02046640007359354, reg_lambda=0.0032197962023686427,
               verbose=-1)

In [20]:
from flaml.ml import sklearn_metric_loss_score

y_pred = automl.predict(X_test)
y_pred_proba = automl.predict_proba(X_test)[:,1]
print('accuracy', '=', 1 - sklearn_metric_loss_score('accuracy', y_pred.astype(int), y_test.astype(int)))
print('roc_auc', '=', 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test.astype(int)))
print('log_loss', '=', sklearn_metric_loss_score('log_loss', y_pred_proba, y_test.astype(int)))

accuracy = 0.9576301615798922
roc_auc = 0.9830828803387144
log_loss = 0.3115500203320831


In [ ]:
performance_mdl = pd.DataFrame(columns = ["Tema", "Área", "Algoritmo", "Accuracy", "ROC_AUC", "Log_Loss"])
performance_mdl = performance_mdl.append({"Tema": "Benefícios", "Área": "Residual - Seguro Defeso - Binário", "Algoritmo": "LGBM",
                                          "Accuracy": 1 - sklearn_metric_loss_score('accuracy', y_pred.astype(int), y_test.astype(int)),
                                          "ROC_AUC": 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test.astype(int)),
                                          "Log_Loss": sklearn_metric_loss_score('log_loss', y_pred_proba, y_test.astype(int))}, ignore_index = True)

performance_mdl.to_sql('performance_mdl', con=conn, if_exists='append', index=False)

In [21]:
import pickle

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_lgbm_beneficios_resid_seguro_defeso_binario.pkl', 'wb') as f:
    pickle.dump(automl, f, pickle.HIGHEST_PROTOCOL)

##### Catboost

In [22]:
from flaml import AutoML

automl = AutoML()
settings = {
    "time_budget": 360,  
    "metric": 'roc_auc', 
    "estimator_list": ['catboost'],  
    "task": 'classification' 
}

automl.fit(X_train = X_train, y_train = y_train, **settings)

[flaml.automl: 09-24 14:00:26] {2600} INFO - task = classification
INFO:flaml.automl:task = classification
[flaml.automl: 09-24 14:00:26] {2602} INFO - Data split method: stratified
INFO:flaml.automl:Data split method: stratified
[flaml.automl: 09-24 14:00:26] {2605} INFO - Evaluation method: holdout
INFO:flaml.automl:Evaluation method: holdout
[flaml.automl: 09-24 14:00:26] {2727} INFO - Minimizing error metric: 1-roc_auc
INFO:flaml.automl:Minimizing error metric: 1-roc_auc
[flaml.automl: 09-24 14:00:26] {2869} INFO - List of ML learners in AutoML Run: ['catboost']
INFO:flaml.automl:List of ML learners in AutoML Run: ['catboost']
[flaml.automl: 09-24 14:00:26] {3174} INFO - iteration 0, current learner catboost
INFO:flaml.automl:iteration 0, current learner catboost
[flaml.automl: 09-24 14:01:08] {3308} INFO - Estimated sufficient time budget=1671884s. Estimated necessary time budget=1672s.
INFO:flaml.automl:Estimated sufficient time budget=1671884s. Estimated necessary time budget=16

In [23]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best accuracy on validation data: {0:.4g}'.format(1-automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

Best ML leaner: catboost
Best hyperparmeter config: {'early_stopping_rounds': 10, 'learning_rate': 0.09999999999999996, 'n_estimators': 94, 'FLAML_sample_size': 40104}
Best accuracy on validation data: 0.9869
Training duration of best run: 21.29 s


In [24]:
automl.model.estimator

In [25]:
from flaml.ml import sklearn_metric_loss_score

y_pred = automl.predict(X_test)
y_pred_proba = automl.predict_proba(X_test)[:,1]
print('accuracy', '=', 1 - sklearn_metric_loss_score('accuracy', y_pred.astype(int), y_test.astype(int)))
print('roc_auc', '=', 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test.astype(int)))
print('log_loss', '=', sklearn_metric_loss_score('log_loss', y_pred_proba, y_test.astype(int)))

accuracy = 0.9513464991023339
roc_auc = 0.9832362969787344
log_loss = 0.14305200383411965


In [ ]:
performance_mdl = pd.DataFrame(columns = ["Tema", "Área", "Algoritmo", "Accuracy", "ROC_AUC", "Log_Loss"])
performance_mdl = performance_mdl.append({"Tema": "Benefícios", "Área": "Residual - Seguro Defeso - Binário", "Algoritmo": "CatBoost",
                                          "Accuracy": 1 - sklearn_metric_loss_score('accuracy', y_pred.astype(int), y_test.astype(int)),
                                          "ROC_AUC": 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test.astype(int)),
                                          "Log_Loss": sklearn_metric_loss_score('log_loss', y_pred_proba, y_test.astype(int))}, ignore_index = True)

performance_mdl.to_sql('performance_mdl', con=conn, if_exists='append', index=False)

In [26]:
import pickle

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_cb_beneficios_resid_seguro_defeso_binario.pkl', 'wb') as f:
    pickle.dump(automl, f, pickle.HIGHEST_PROTOCOL)

##### Ensemble

In [9]:
from sklearn.linear_model import LogisticRegression
from flaml.ml import sklearn_metric_loss_score
import pickle

pred_train_ensem = pd.DataFrame()
pred_test_ensem = pd.DataFrame()

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_xgb_beneficios_resid_seguro_defeso_binario.pkl', 'rb') as f:
    mdl_xgb = pickle.load(f)

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_lgbm_beneficios_resid_seguro_defeso_binario.pkl', 'rb') as f:
    mdl_lgbm = pickle.load(f)

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_cb_beneficios_resid_seguro_defeso_binario.pkl', 'rb') as f:
    mdl_cb = pickle.load(f)

pred_train_ensem[["xgb_proba_0_resid_seguro_defeso_binario", "xgb_proba_1_resid_seguro_defeso_binario"]] = mdl_xgb.predict_proba(X_train)
pred_test_ensem[["xgb_proba_0_resid_seguro_defeso_binario", "xgb_proba_1_resid_seguro_defeso_binario"]] = mdl_xgb.predict_proba(X_test)
pred_train_ensem[["lgbm_proba_0_resid_seguro_defeso_binario", "lgbm_proba_1_resid_seguro_defeso_binario"]] = mdl_lgbm.predict_proba(X_train)
pred_test_ensem[["lgbm_proba_0_resid_seguro_defeso_binario", "lgbm_proba_1_resid_seguro_defeso_binario"]] = mdl_lgbm.predict_proba(X_test)
pred_train_ensem[["cb_proba_0_resid_seguro_defeso_binario", "cb_proba_1_resid_seguro_defeso_binario"]] = mdl_cb.predict_proba(X_train)
pred_test_ensem[["cb_proba_0_resid_seguro_defeso_binario", "cb_proba_1_resid_seguro_defeso_binario"]] = mdl_cb.predict_proba(X_test)

pred_train_ensem["y_train"] = y_train.reset_index(drop=True)
pred_test_ensem["y_test"] = y_test.reset_index(drop=True)

X_train_ensem = pred_train_ensem.drop(["y_train"], 1)
X_test_ensem = pred_test_ensem.drop(["y_test"], 1)
y_train_ensem = pred_train_ensem["y_train"]
y_test_ensem = pred_test_ensem["y_test"]

clf = LogisticRegression().fit(X_train_ensem, y_train_ensem.astype(int))
y_pred = clf.predict(X_test_ensem)
y_pred_proba = clf.predict_proba(X_test_ensem)[:,1]
print('accuracy', '=', 1 - sklearn_metric_loss_score('accuracy', y_pred.astype(int), y_test.astype(int)))
print('roc_auc', '=', 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test.astype(int)))
print('log_loss', '=', sklearn_metric_loss_score('log_loss', y_pred_proba, y_test.astype(int)))

performance_mdl = pd.DataFrame(columns = ["Tema", "Área", "Algoritmo", "Accuracy", "ROC_AUC", "Log_Loss"])
performance_mdl = performance_mdl.append({"Tema": "Benefícios", "Área": "Residual - Seguro Defeso - Binário", "Algoritmo": "Ensemble",
                                          "Accuracy": 1 - sklearn_metric_loss_score('accuracy', y_pred.astype(int), y_test.astype(int)),
                                          "ROC_AUC": 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test.astype(int)),
                                          "Log_Loss": sklearn_metric_loss_score('log_loss', y_pred_proba, y_test.astype(int))}, ignore_index = True)

performance_mdl.to_sql('performance_mdl', con=conn, if_exists='append', index=False)

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/transferencias/mdl_ensem_beneficios_resid_seguro_defeso_binario.pkl', 'wb') as f:
    pickle.dump(clf, f, pickle.HIGHEST_PROTOCOL)

preds_beneficios = pd.DataFrame()

X_preds = agg_municipios.drop(["codigo_ibge", "nome", "siafi_id"], 1)
X_preds["ano"] = 2022

preds_beneficios[["xgb_proba_0_resid_seguro_defeso_binario", "xgb_proba_1_resid_seguro_defeso_binario"]] = mdl_xgb.predict_proba(X_preds)
preds_beneficios[["lgbm_proba_0_resid_seguro_defeso_binario", "lgbm_proba_1_resid_seguro_defeso_binario"]] = mdl_lgbm.predict_proba(X_preds)
preds_beneficios[["cb_proba_0_resid_seguro_defeso_binario", "cb_proba_1_resid_seguro_defeso_binario"]] = mdl_cb.predict_proba(X_preds)

preds_beneficios[["candido_pred_proba_0_resid_seguro_defeso_binario", "candido_pred_proba_1_resid_seguro_defeso_binario"]] = clf.predict_proba(preds_beneficios)
preds_beneficios["candido_pred_seguro_defeso_residual_binario"] = clf.predict(preds_beneficios.drop(["codigo_ibge", "siafi_id", "nome", "candido_pred_proba_0_resid_seguro_defeso_binario", "candido_pred_proba_1_resid_seguro_defeso_binario"], 1))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


accuracy = 0.9576301615798922
roc_auc = 0.9848749397150434
log_loss = 0.2972638315420175


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


KeyError: ignored

In [11]:
preds_beneficios = pd.DataFrame()

X_preds = agg_municipios.drop(["codigo_ibge", "nome", "siafi_id"], 1)
X_preds["ano"] = 2022

preds_beneficios[["xgb_proba_0_resid_seguro_defeso_binario", "xgb_proba_1_resid_seguro_defeso_binario"]] = mdl_xgb.predict_proba(X_preds)
preds_beneficios[["lgbm_proba_0_resid_seguro_defeso_binario", "lgbm_proba_1_resid_seguro_defeso_binario"]] = mdl_lgbm.predict_proba(X_preds)
preds_beneficios[["cb_proba_0_resid_seguro_defeso_binario", "cb_proba_1_resid_seguro_defeso_binario"]] = mdl_cb.predict_proba(X_preds)

preds_beneficios[["candido_pred_proba_0_resid_seguro_defeso_binario", "candido_pred_proba_1_resid_seguro_defeso_binario"]] = clf.predict_proba(preds_beneficios)
preds_beneficios["candido_pred_seguro_defeso_residual_binario"] = clf.predict(preds_beneficios.drop(["candido_pred_proba_0_resid_seguro_defeso_binario", "candido_pred_proba_1_resid_seguro_defeso_binario"], 1))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  # This is added back by InteractiveShellApp.init_path()


In [12]:
preds_beneficios

,xgb_proba_0_resid_seguro_defeso_binario,xgb_proba_1_resid_seguro_defeso_binario,lgbm_proba_0_resid_seguro_defeso_binario,lgbm_proba_1_resid_seguro_defeso_binario,cb_proba_0_resid_seguro_defeso_binario,cb_proba_1_resid_seguro_defeso_binario,candido_pred_proba_0_resid_seguro_defeso_binario,candido_pred_proba_1_resid_seguro_defeso_binario,candido_pred_seguro_defeso_residual_binario
0,0.673500,0.326500,9.990044e-01,9.956380e-04,0.960964,0.039036,0.999856,0.000144,0
1,0.670030,0.329970,9.998427e-01,1.572885e-04,0.985802,0.014198,0.999870,0.000130,0
2,0.665333,0.334667,9.997968e-01,2.031953e-04,0.950345,0.049655,0.999851,0.000149,0
3,0.666696,0.333304,9.988937e-01,1.106283e-03,0.984992,0.015008,0.999867,0.000133,0
4,0.647511,0.352489,9.987699e-01,1.230063e-03,0.945741,0.054259,0.999843,0.000157,0
...,...,...,...,...,...,...,...,...,...
5565,0.329209,0.670791,1.688717e-06,9.999983e-01,0.032113,0.967887,0.000198,0.999802,1
5566,0.382273,0.617727,1.229705e-04,9.998770e-01,0.044991,0.955009,0.000223,0.999777,1
5567,0.639369,0.360631,9.993512e-01,6.487658e-04,0.810371,0.189629,0.999739,0.000261,0
5568,0.333400,0.666600,5.860315e-07,9.999994e-01,0.007871,0.992129,0.000182,0.999818,1


In [13]:
preds_beneficios_db = pd.read_sql('select * from preds_beneficios', con = conn)
preds_beneficios_db = pd.concat([preds_beneficios_db, preds_beneficios], axis = 1)
preds_beneficios_db.to_sql('preds_beneficios', con=conn, if_exists='replace', index=False)

### PETI - Erradicação do Trabalho Infantil

#### Binário

In [14]:
peti_ = peti[peti["valor_parcela_agg"]!=0]
#peti_["siafi_id"] = peti_["siafi_id"].str.replace(".0", "", regex = True).astype(str)
peti_ = peti_.reset_index(drop=True)
peti_["ano"] = peti_["ano_mes"].astype(str).str.slice(0, 4)
peti_["mes"] = peti_["ano_mes"].astype(str).str.slice(4, 6)
peti_ = peti_[["siafi_id", "ano", "valor_parcela_agg"]].groupby(["siafi_id", "ano"]).sum().reset_index()
peti_ = peti_[peti_["siafi_id"]!=""][peti_["siafi_id"]!="0"].astype(int)

In [15]:
list_cols = agg_municipios.columns.to_list()
list_cols.append("target")
list_cols.append("ano")
dataset = pd.DataFrame(columns = list_cols)

for a in peti_["ano"].unique():
  target = peti_[peti_["ano"]==a]
  target["target"] = 1
  target = pd.merge(agg_municipios, target[["siafi_id", "target"]], how = "left", on = "siafi_id")
  target["target"] = target["target"].fillna(0).astype(int)
  target["ano"] = a
  dataset = dataset.append(target, ignore_index=True)

dataset["ano"] = dataset["ano"].astype(int)
dataset = dataset.drop_duplicates()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
X_train = dataset[dataset["ano"]!=2021].drop(["codigo_ibge", "nome", "siafi_id", "target"], 1)
y_train = dataset[dataset["ano"]!=2021]["target"]
X_test = dataset[dataset["ano"]==2021].drop(["codigo_ibge", "nome", "siafi_id", "target"], 1)
y_test = dataset[dataset["ano"]==2021]["target"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


##### XGBoost

In [ ]:
from flaml import AutoML

automl = AutoML()
settings = {
    "time_budget": 360,  
    "metric": 'roc_auc', 
    "estimator_list": ['xgboost'],  
    "task": 'classification' 
}

automl.fit(X_train = X_train, y_train = y_train, **settings)

[flaml.automl: 09-16 18:49:34] {2600} INFO - task = classification
INFO:flaml.automl:task = classification
[flaml.automl: 09-16 18:49:34] {2602} INFO - Data split method: stratified
INFO:flaml.automl:Data split method: stratified
[flaml.automl: 09-16 18:49:34] {2605} INFO - Evaluation method: holdout
INFO:flaml.automl:Evaluation method: holdout
[flaml.automl: 09-16 18:49:34] {2727} INFO - Minimizing error metric: 1-roc_auc
INFO:flaml.automl:Minimizing error metric: 1-roc_auc
[flaml.automl: 09-16 18:49:34] {2869} INFO - List of ML learners in AutoML Run: ['xgboost']
INFO:flaml.automl:List of ML learners in AutoML Run: ['xgboost']
[flaml.automl: 09-16 18:49:34] {3174} INFO - iteration 0, current learner xgboost
INFO:flaml.automl:iteration 0, current learner xgboost
[flaml.automl: 09-16 18:49:36] {3308} INFO - Estimated sufficient time budget=62813s. Estimated necessary time budget=63s.
INFO:flaml.automl:Estimated sufficient time budget=62813s. Estimated necessary time budget=63s.
[flaml.

In [ ]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best accuracy on validation data: {0:.4g}'.format(1-automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

Best ML leaner: xgboost
Best hyperparmeter config: {'n_estimators': 304, 'max_leaves': 18, 'min_child_weight': 26.122315725347097, 'learning_rate': 1.0, 'subsample': 0.7499627791476885, 'colsample_bylevel': 0.6922891497872719, 'colsample_bytree': 0.5128239583647158, 'reg_alpha': 21.8623140638164, 'reg_lambda': 1.029992313726939, 'FLAML_sample_size': 40104}
Best accuracy on validation data: 0.9836
Training duration of best run: 38.39 s


In [ ]:
automl.model.estimator

XGBClassifier(colsample_bylevel=0.6922891497872719,
              colsample_bytree=0.5128239583647158, grow_policy='lossguide',
              learning_rate=1.0, max_depth=0, max_leaves=18,
              min_child_weight=26.122315725347097, n_estimators=304, n_jobs=-1,
              reg_alpha=21.8623140638164, reg_lambda=1.029992313726939,
              subsample=0.7499627791476885, tree_method='hist',
              use_label_encoder=False, verbosity=0)

In [ ]:
from flaml.ml import sklearn_metric_loss_score

y_pred = automl.predict(X_test)
y_pred_proba = automl.predict_proba(X_test)[:,1]
print('accuracy', '=', 1 - sklearn_metric_loss_score('accuracy', y_pred.astype(int), y_test.astype(int)))
print('roc_auc', '=', 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test.astype(int)))
print('log_loss', '=', sklearn_metric_loss_score('log_loss', y_pred_proba, y_test.astype(int)))

accuracy = 0.9940754039497307
roc_auc = 0.9936171080141402
log_loss = 0.01957123508413255


In [ ]:
performance_mdl = pd.DataFrame(columns = ["Tema", "Área", "Algoritmo", "Accuracy", "ROC_AUC", "Log_Loss"])
performance_mdl = performance_mdl.append({"Tema": "Benefícios", "Área": "Residual - PETI - Binário", "Algoritmo": "XGBoost",
                                          "Accuracy": 1 - sklearn_metric_loss_score('accuracy', y_pred.astype(int), y_test.astype(int)),
                                          "ROC_AUC": 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test.astype(int)),
                                          "Log_Loss": sklearn_metric_loss_score('log_loss', y_pred_proba, y_test.astype(int))}, ignore_index = True)

performance_mdl.to_sql('performance_mdl', con=conn, if_exists='append', index=False)

In [ ]:
import pickle

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_xgb_beneficios_resid_peti_binario.pkl', 'wb') as f:
    pickle.dump(automl, f, pickle.HIGHEST_PROTOCOL)

##### LGBM

In [ ]:
from flaml import AutoML

automl = AutoML()
settings = {
    "time_budget": 360,  
    "metric": 'roc_auc', 
    "estimator_list": ['lgbm'],  
    "task": 'classification' 
}

automl.fit(X_train = X_train, y_train = y_train, **settings)

[flaml.automl: 09-16 18:57:21] {2600} INFO - task = classification
INFO:flaml.automl:task = classification
[flaml.automl: 09-16 18:57:21] {2602} INFO - Data split method: stratified
INFO:flaml.automl:Data split method: stratified
[flaml.automl: 09-16 18:57:21] {2605} INFO - Evaluation method: holdout
INFO:flaml.automl:Evaluation method: holdout
[flaml.automl: 09-16 18:57:21] {2727} INFO - Minimizing error metric: 1-roc_auc
INFO:flaml.automl:Minimizing error metric: 1-roc_auc
[flaml.automl: 09-16 18:57:21] {2869} INFO - List of ML learners in AutoML Run: ['lgbm']
INFO:flaml.automl:List of ML learners in AutoML Run: ['lgbm']
[flaml.automl: 09-16 18:57:21] {3174} INFO - iteration 0, current learner lgbm
INFO:flaml.automl:iteration 0, current learner lgbm
[flaml.automl: 09-16 18:57:22] {3308} INFO - Estimated sufficient time budget=29096s. Estimated necessary time budget=29s.
INFO:flaml.automl:Estimated sufficient time budget=29096s. Estimated necessary time budget=29s.
[flaml.automl: 09-1

In [ ]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best accuracy on validation data: {0:.4g}'.format(1-automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

Best ML leaner: lgbm
Best hyperparmeter config: {'n_estimators': 306, 'num_leaves': 54, 'min_child_samples': 3, 'learning_rate': 0.07020082747656131, 'log_max_bin': 10, 'colsample_bytree': 0.391948894699114, 'reg_alpha': 0.032619809462956464, 'reg_lambda': 0.0020939954052258724, 'FLAML_sample_size': 40104}
Best accuracy on validation data: 0.9911
Training duration of best run: 66.79 s


In [ ]:
automl.model.estimator

LGBMClassifier(colsample_bytree=0.391948894699114,
               learning_rate=0.07020082747656131, max_bin=1023,
               min_child_samples=3, n_estimators=306, num_leaves=54,
               reg_alpha=0.032619809462956464, reg_lambda=0.0020939954052258724,
               verbose=-1)

In [ ]:
from flaml.ml import sklearn_metric_loss_score

y_pred = automl.predict(X_test)
y_pred_proba = automl.predict_proba(X_test)[:,1]
print('accuracy', '=', 1 - sklearn_metric_loss_score('accuracy', y_pred.astype(int), y_test.astype(int)))
print('roc_auc', '=', 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test.astype(int)))
print('log_loss', '=', sklearn_metric_loss_score('log_loss', y_pred_proba, y_test.astype(int)))

accuracy = 0.9956912028725314
roc_auc = 0.9980432009970357
log_loss = 0.015070946460748852


In [ ]:
performance_mdl = pd.DataFrame(columns = ["Tema", "Área", "Algoritmo", "Accuracy", "ROC_AUC", "Log_Loss"])
performance_mdl = performance_mdl.append({"Tema": "Benefícios", "Área": "Residual - PETI - Binário", "Algoritmo": "LGBM",
                                          "Accuracy": 1 - sklearn_metric_loss_score('accuracy', y_pred.astype(int), y_test.astype(int)),
                                          "ROC_AUC": 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test.astype(int)),
                                          "Log_Loss": sklearn_metric_loss_score('log_loss', y_pred_proba, y_test.astype(int))}, ignore_index = True)

performance_mdl.to_sql('performance_mdl', con=conn, if_exists='append', index=False)

In [ ]:
import pickle

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_lgbm_beneficios_resid_peti_binario.pkl', 'wb') as f:
    pickle.dump(automl, f, pickle.HIGHEST_PROTOCOL)

##### CatBoost

In [ ]:
from flaml import AutoML

automl = AutoML()
settings = {
    "time_budget": 360,  
    "metric": 'roc_auc', 
    "estimator_list": ['lgbm'],  
    "task": 'classification' 
}

automl.fit(X_train = X_train, y_train = y_train, **settings)

[flaml.automl: 09-16 19:03:32] {2600} INFO - task = classification
INFO:flaml.automl:task = classification
[flaml.automl: 09-16 19:03:32] {2602} INFO - Data split method: stratified
INFO:flaml.automl:Data split method: stratified
[flaml.automl: 09-16 19:03:32] {2605} INFO - Evaluation method: holdout
INFO:flaml.automl:Evaluation method: holdout
[flaml.automl: 09-16 19:03:32] {2727} INFO - Minimizing error metric: 1-roc_auc
INFO:flaml.automl:Minimizing error metric: 1-roc_auc
[flaml.automl: 09-16 19:03:32] {2869} INFO - List of ML learners in AutoML Run: ['lgbm']
INFO:flaml.automl:List of ML learners in AutoML Run: ['lgbm']
[flaml.automl: 09-16 19:03:32] {3174} INFO - iteration 0, current learner lgbm
INFO:flaml.automl:iteration 0, current learner lgbm
[flaml.automl: 09-16 19:03:33] {3308} INFO - Estimated sufficient time budget=28829s. Estimated necessary time budget=29s.
INFO:flaml.automl:Estimated sufficient time budget=28829s. Estimated necessary time budget=29s.
[flaml.automl: 09-1

In [ ]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best accuracy on validation data: {0:.4g}'.format(1-automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

Best ML leaner: lgbm
Best hyperparmeter config: {'n_estimators': 306, 'num_leaves': 54, 'min_child_samples': 3, 'learning_rate': 0.07020082747656131, 'log_max_bin': 10, 'colsample_bytree': 0.391948894699114, 'reg_alpha': 0.032619809462956464, 'reg_lambda': 0.0020939954052258724, 'FLAML_sample_size': 40104}
Best accuracy on validation data: 0.9911
Training duration of best run: 78.92 s


In [ ]:
automl.model.estimator

LGBMClassifier(colsample_bytree=0.391948894699114,
               learning_rate=0.07020082747656131, max_bin=1023,
               min_child_samples=3, n_estimators=306, num_leaves=54,
               reg_alpha=0.032619809462956464, reg_lambda=0.0020939954052258724,
               verbose=-1)

In [ ]:
from flaml.ml import sklearn_metric_loss_score

y_pred = automl.predict(X_test)
y_pred_proba = automl.predict_proba(X_test)[:,1]
print('accuracy', '=', 1 - sklearn_metric_loss_score('accuracy', y_pred.astype(int), y_test.astype(int)))
print('roc_auc', '=', 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test.astype(int)))
print('log_loss', '=', sklearn_metric_loss_score('log_loss', y_pred_proba, y_test.astype(int)))

accuracy = 0.9956912028725314
roc_auc = 0.9980432009970357
log_loss = 0.015070946460748852


In [ ]:
performance_mdl = pd.DataFrame(columns = ["Tema", "Área", "Algoritmo", "Accuracy", "ROC_AUC", "Log_Loss"])
performance_mdl = performance_mdl.append({"Tema": "Benefícios", "Área": "Residual - PETI - Binário", "Algoritmo": "CatBoost",
                                          "Accuracy": 1 - sklearn_metric_loss_score('accuracy', y_pred.astype(int), y_test.astype(int)),
                                          "ROC_AUC": 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test.astype(int)),
                                          "Log_Loss": sklearn_metric_loss_score('log_loss', y_pred_proba, y_test.astype(int))}, ignore_index = True)

performance_mdl.to_sql('performance_mdl', con=conn, if_exists='append', index=False)

In [ ]:
import pickle

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_cb_beneficios_resid_peti_binario.pkl', 'wb') as f:
    pickle.dump(automl, f, pickle.HIGHEST_PROTOCOL)

##### Ensemble

In [17]:
from sklearn.linear_model import LogisticRegression
from flaml.ml import sklearn_metric_loss_score
import pickle

pred_train_ensem = pd.DataFrame()
pred_test_ensem = pd.DataFrame()

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_xgb_beneficios_resid_peti_binario.pkl', 'rb') as f:
    mdl_xgb = pickle.load(f)

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_lgbm_beneficios_resid_peti_binario.pkl', 'rb') as f:
    mdl_lgbm = pickle.load(f)

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_cb_beneficios_resid_peti_binario.pkl', 'rb') as f:
    mdl_cb = pickle.load(f)

pred_train_ensem[["xgb_proba_0_peti_residual_binario", "xgb_proba_1_peti_residual_binario"]] = mdl_xgb.predict_proba(X_train)
pred_test_ensem[["xgb_proba_0_peti_residual_binario", "xgb_proba_1_peti_residual_binario"]] = mdl_xgb.predict_proba(X_test)
pred_train_ensem[["lgbm_proba_0_peti_residual_binario", "lgbm_proba_1_peti_residual_binario"]] = mdl_lgbm.predict_proba(X_train)
pred_test_ensem[["lgbm_proba_0_peti_residual_binario", "lgbm_proba_1_peti_residual_binario"]] = mdl_lgbm.predict_proba(X_test)
pred_train_ensem[["cb_proba_0_peti_residual_binario", "cb_proba_1_peti_residual_binario"]] = mdl_cb.predict_proba(X_train)
pred_test_ensem[["cb_proba_0_peti_residual_binario", "cb_proba_1_peti_residual_binario"]] = mdl_cb.predict_proba(X_test)

pred_train_ensem["y_train"] = y_train.reset_index(drop=True)
pred_test_ensem["y_test"] = y_test.reset_index(drop=True)

X_train_ensem = pred_train_ensem.drop(["y_train"], 1)
X_test_ensem = pred_test_ensem.drop(["y_test"], 1)
y_train_ensem = pred_train_ensem["y_train"]
y_test_ensem = pred_test_ensem["y_test"]

clf = LogisticRegression().fit(X_train_ensem, y_train_ensem.astype(int))
y_pred = clf.predict(X_test_ensem)
y_pred_proba = clf.predict_proba(X_test_ensem)[:,1]
print('accuracy', '=', 1 - sklearn_metric_loss_score('accuracy', y_pred.astype(int), y_test.astype(int)))
print('roc_auc', '=', 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test.astype(int)))
print('log_loss', '=', sklearn_metric_loss_score('log_loss', y_pred_proba, y_test.astype(int)))

performance_mdl = pd.DataFrame(columns = ["Tema", "Área", "Algoritmo", "Accuracy", "ROC_AUC", "Log_Loss"])
performance_mdl = performance_mdl.append({"Tema": "Benefícios", "Área": "Residual - PETI - Binário", "Algoritmo": "Ensemble",
                                          "Accuracy": 1 - sklearn_metric_loss_score('accuracy', y_pred.astype(int), y_test.astype(int)),
                                          "ROC_AUC": 1 - sklearn_metric_loss_score('roc_auc', y_pred_proba, y_test.astype(int)),
                                          "Log_Loss": sklearn_metric_loss_score('log_loss', y_pred_proba, y_test.astype(int))}, ignore_index = True)

performance_mdl.to_sql('performance_mdl', con=conn, if_exists='append', index=False)

preds_beneficios = pd.DataFrame()

X_preds = agg_municipios.drop(["codigo_ibge", "nome", "siafi_id"], 1)
X_preds["ano"] = 2022

preds_beneficios[["xgb_proba_0_resid_peti_binario", "xgb_proba_1_resid_peti_binario"]] = mdl_xgb.predict_proba(X_preds)
preds_beneficios[["lgbm_proba_0_resid_peti_binario", "lgbm_proba_1_resid_peti_binario"]] = mdl_lgbm.predict_proba(X_preds)
preds_beneficios[["cb_proba_0_resid_peti_binario", "cb_proba_1_resid_peti_binario"]] = mdl_cb.predict_proba(X_preds)

preds_beneficios[["candido_pred_proba_0_resid_peti_binario", "candido_pred_proba_1_resid_peti_binario"]] = clf.predict_proba(preds_beneficios)
preds_beneficios["candido_pred_peti_binario"] = clf.predict(preds_beneficios.drop(["candido_pred_proba_0_resid_peti_binario", "candido_pred_proba_1_resid_peti_binario"], 1))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


accuracy = 0.9956912028725314
roc_auc = 0.9973501680168191
log_loss = 0.03387865815864834


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- cb_proba_0_resid_peti_binario
- cb_proba_1_resid_peti_binario
- lgbm_proba_0_resid_peti_binario
- lgbm_proba_1_resid_peti_binario
- xgb_proba_0_resid_peti_binario
- ...
Feature names seen at fit time, yet now missing:
- cb_proba_0_peti_residual_binario
- cb_proba_1_peti_residual_binario
- lgbm_proba_0_peti_residual_binario
- lgbm_proba_1_peti_residual_binario
- xgb_proba_0_peti_residual_binario
- ...

  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: FutureWarning: In a future version of pandas all arguments of DataFram

In [19]:
preds_beneficios_db = pd.read_sql('select * from preds_beneficios', con = conn)
preds_beneficios_db = pd.concat([preds_beneficios_db, preds_beneficios], axis = 1)
preds_beneficios_db.to_sql('preds_beneficios', con=conn, if_exists='replace', index=False)

### BPC

#### Valor Per Capta

In [21]:
dados_agg = pd.DataFrame(columns = ["ano_mes", "siafi_id", "cont_parcelas",	"valor_parcela_agg"])
dados_agg = pd.concat([dados_agg, bpc])
dados_agg["siafi_id"] = dados_agg["siafi_id"].astype(float).astype(int).astype(str)
dados_agg["ano"] = dados_agg["ano_mes"].astype(str).str.slice(0, 4)
dados_agg = dados_agg[["siafi_id", "ano", "valor_parcela_agg"]].groupby(["ano", "siafi_id"]).sum().reset_index()

In [22]:
df_melt = agg_municipios[["siafi_id", "pop_2014", "pop_2015", "pop_2016", "pop_2017", "pop_2018", "pop_2019", "pop_2020", "pop_2021"]]
pop = pd.melt(df_melt, id_vars=["siafi_id"], value_vars=["pop_2014", "pop_2015", "pop_2016", "pop_2017", "pop_2018", "pop_2019", "pop_2020", "pop_2021"])
pop["variable"] = pop["variable"].replace("pop_", "", regex = True)
pop = pop.rename(columns = {"variable": "ano", "value": "pop"})
pop["siafi_id"] = pop["siafi_id"].astype(str)

In [23]:
dados_agg = pd.merge(dados_agg, pop, on = ["siafi_id", "ano"], how = "left")
dados_agg = dados_agg.dropna(subset = ["pop"])
dados_agg["pop"] = dados_agg["pop"].astype(int)
dados_agg["valor_parcela_per_capta"] = dados_agg["valor_parcela_agg"] / dados_agg["pop"]
dados_agg = dados_agg.round({'valor_parcela_per_capta': 2})

In [24]:
list_cols = agg_municipios.columns.tolist()
list_cols.append("ano")
list_cols.append("valor_parcela_per_capta")
dataset = pd.DataFrame(columns = list_cols)

dados_agg = dados_agg.reset_index(drop=True)
dados_agg = dados_agg[["ano", "siafi_id", "valor_parcela_per_capta"]]
agg_municipios["siafi_id"] = agg_municipios["siafi_id"].astype(str)

for ano in range(2014, 2022):
  ds_s_value = dados_agg[dados_agg["ano"]==str(ano)].reset_index(drop=True)
  ds_s_value = pd.merge(ds_s_value, agg_municipios, how = "left", on = "siafi_id") 

  ds_n_value = agg_municipios[~agg_municipios['siafi_id'].isin(ds_s_value["siafi_id"].unique().tolist())]
  ds_n_value["ano"] = str(ano)
  ds_n_value["valor_parcela_per_capta"] = 0
  ds_n_value = ds_n_value.reset_index(drop=True)

  dataset = pd.concat([dataset, pd.concat([ds_n_value, ds_s_value], ignore_index = True)], ignore_index = True)

dataset["ano"] = dataset["ano"].astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [25]:
X_train = dataset[(dataset["ano"]!=2021)].drop(["codigo_ibge", "nome", "siafi_id", "valor_parcela_per_capta"], 1)
y_train = dataset[(dataset["ano"]!=2021)]["valor_parcela_per_capta"]
X_test = dataset[dataset["ano"]==2021].drop(["codigo_ibge", "nome", "siafi_id", "valor_parcela_per_capta"], 1)
y_test = dataset[dataset["ano"]==2021]["valor_parcela_per_capta"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


##### XGBoost

In [26]:
from flaml import AutoML

automl = AutoML()
settings = {
    "time_budget": 720,  
    "metric": 'r2', 
    "estimator_list": ['xgboost'],  
    "task": 'regression'    
}

automl.fit(X_train = X_train, y_train = y_train, **settings)

[flaml.automl: 09-25 04:32:49] {2600} INFO - task = regression
INFO:flaml.automl:task = regression
[flaml.automl: 09-25 04:32:49] {2602} INFO - Data split method: uniform
INFO:flaml.automl:Data split method: uniform
[flaml.automl: 09-25 04:32:49] {2605} INFO - Evaluation method: holdout
INFO:flaml.automl:Evaluation method: holdout
[flaml.automl: 09-25 04:32:49] {2727} INFO - Minimizing error metric: 1-r2
INFO:flaml.automl:Minimizing error metric: 1-r2
[flaml.automl: 09-25 04:32:49] {2869} INFO - List of ML learners in AutoML Run: ['xgboost']
INFO:flaml.automl:List of ML learners in AutoML Run: ['xgboost']
[flaml.automl: 09-25 04:32:49] {3174} INFO - iteration 0, current learner xgboost
INFO:flaml.automl:iteration 0, current learner xgboost
[flaml.automl: 09-25 04:32:51] {3308} INFO - Estimated sufficient time budget=24416s. Estimated necessary time budget=24s.
INFO:flaml.automl:Estimated sufficient time budget=24416s. Estimated necessary time budget=24s.
[flaml.automl: 09-25 04:32:51] 

In [27]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

Best ML leaner: xgboost
Best hyperparmeter config: {'n_estimators': 206, 'max_leaves': 717, 'min_child_weight': 3.1384719243236003, 'learning_rate': 0.09177721138536776, 'subsample': 0.8413048297641477, 'colsample_bylevel': 0.5402300960155633, 'colsample_bytree': 0.6718697198293443, 'reg_alpha': 0.02426727770924963, 'reg_lambda': 0.13688322947997558}
Training duration of best run: 70.26 s


In [28]:
automl.model.estimator

XGBRegressor(colsample_bylevel=0.5402300960155633,
             colsample_bytree=0.6718697198293443, grow_policy='lossguide',
             learning_rate=0.09177721138536776, max_depth=0, max_leaves=717,
             min_child_weight=3.1384719243236003, n_estimators=104, n_jobs=-1,
             reg_alpha=0.02426727770924963, reg_lambda=0.13688322947997558,
             subsample=0.8413048297641477, tree_method='hist',
             use_label_encoder=False, verbosity=0)

In [29]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

y_pred = automl.predict(X_test)

print('r2', '=', r2_score(y_test, y_pred))
print('mse', '=', mean_squared_error(y_test, y_pred))
print('rmse', '=', mean_squared_error(y_test, y_pred, squared=False))

r2 = 0.9332577891006917
mse = 1820.1780709111645
rmse = 42.66354498762573


In [ ]:
performance_mdl_reg = pd.DataFrame(columns = ["Tema", "Área", "Algoritmo", "R2", "MSE", "RMSE"])
performance_mdl_reg = performance_mdl_reg.append({"Tema": "Benefícios", "Área": "Residual - BPC - Valor Per Capta", "Algoritmo": "XGBoost",
                                          "R2": r2_score(y_test, y_pred),
                                          "MSE": mean_squared_error(y_test, y_pred),
                                          "RMSE": mean_squared_error(y_test, y_pred, squared=False)}, ignore_index = True)

performance_mdl_reg.to_sql('performance_mdl_reg', con=conn, if_exists='append', index=False)

In [30]:
import pickle

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_xgb_beneficios_resid_bpc_valor_per_capta.pkl', 'wb') as f:
    pickle.dump(automl, f, pickle.HIGHEST_PROTOCOL)

##### LGBM

In [37]:
from flaml import AutoML

automl = AutoML()
settings = {
    "time_budget": 360,  
    "metric": 'r2', 
    "estimator_list": ['lgbm'],  
    "task": 'regression'    
}

automl.fit(X_train = X_train, y_train = y_train, **settings)

[flaml.automl: 09-25 05:04:27] {2600} INFO - task = regression
INFO:flaml.automl:task = regression
[flaml.automl: 09-25 05:04:27] {2602} INFO - Data split method: uniform
INFO:flaml.automl:Data split method: uniform
[flaml.automl: 09-25 05:04:27] {2605} INFO - Evaluation method: holdout
INFO:flaml.automl:Evaluation method: holdout
[flaml.automl: 09-25 05:04:27] {2727} INFO - Minimizing error metric: 1-r2
INFO:flaml.automl:Minimizing error metric: 1-r2
[flaml.automl: 09-25 05:04:27] {2869} INFO - List of ML learners in AutoML Run: ['lgbm']
INFO:flaml.automl:List of ML learners in AutoML Run: ['lgbm']
[flaml.automl: 09-25 05:04:27] {3174} INFO - iteration 0, current learner lgbm
INFO:flaml.automl:iteration 0, current learner lgbm
[flaml.automl: 09-25 05:04:29] {3308} INFO - Estimated sufficient time budget=23338s. Estimated necessary time budget=23s.
INFO:flaml.automl:Estimated sufficient time budget=23338s. Estimated necessary time budget=23s.
[flaml.automl: 09-25 05:04:29] {3360} INFO 

In [38]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

Best ML leaner: lgbm
Best hyperparmeter config: {'n_estimators': 88, 'num_leaves': 493, 'min_child_samples': 2, 'learning_rate': 0.35819596846132973, 'log_max_bin': 10, 'colsample_bytree': 0.6146637675541853, 'reg_alpha': 0.003934614746573571, 'reg_lambda': 0.0018340820859991315}
Training duration of best run: 54.96 s


In [41]:
automl.model.estimator

LGBMRegressor(colsample_bytree=0.6146637675541853,
              learning_rate=0.35819596846132973, max_bin=1023,
              min_child_samples=2, n_estimators=88, num_leaves=493,
              reg_alpha=0.003934614746573571, reg_lambda=0.0018340820859991315,
              verbose=-1)

In [40]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

y_pred = automl.predict(X_test)

print('r2', '=', r2_score(y_test, y_pred))
print('mse', '=', mean_squared_error(y_test, y_pred))
print('rmse', '=', mean_squared_error(y_test, y_pred, squared=False))

r2 = 0.9379049743529411
mse = 1693.4411142891015
rmse = 41.151441217642684


In [42]:
performance_mdl_reg = pd.DataFrame(columns = ["Tema", "Área", "Algoritmo", "R2", "MSE", "RMSE"])
performance_mdl_reg = performance_mdl_reg.append({"Tema": "Benefícios", "Área": "Residual - BPC - Valor Per Capta", "Algoritmo": "LGBM",
                                          "R2": r2_score(y_test, y_pred),
                                          "MSE": mean_squared_error(y_test, y_pred),
                                          "RMSE": mean_squared_error(y_test, y_pred, squared=False)}, ignore_index = True)

performance_mdl_reg.to_sql('performance_mdl_reg', con=conn, if_exists='append', index=False)

In [43]:
import pickle

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_lgbm_beneficios_resid_bpc_valor_per_capta.pkl', 'wb') as f:
    pickle.dump(automl, f, pickle.HIGHEST_PROTOCOL)

##### CatBoost

In [31]:
from flaml import AutoML

automl = AutoML()
settings = {
    "time_budget": 720,  
    "metric": 'r2', 
    "estimator_list": ['catboost'],  
    "task": 'regression'    
}

automl.fit(X_train = X_train, y_train = y_train, **settings)

[flaml.automl: 09-25 04:46:32] {2727} INFO - Minimizing error metric: 1-r2
INFO:flaml.automl:Minimizing error metric: 1-r2
[flaml.automl: 09-25 04:46:32] {2869} INFO - List of ML learners in AutoML Run: ['catboost']
INFO:flaml.automl:List of ML learners in AutoML Run: ['catboost']
[flaml.automl: 09-25 04:46:32] {3174} INFO - iteration 0, current learner catboost
INFO:flaml.automl:iteration 0, current learner catboost
[flaml.automl: 09-25 04:51:01] {3308} INFO - Estimated sufficient time budget=2683621s. Estimated necessary time budget=2684s.
INFO:flaml.automl:Estimated sufficient time budget=2683621s. Estimated necessary time budget=2684s.
[flaml.automl: 09-25 04:51:01] {3360} INFO -  at 271.9s,	estimator catboost's best error=0.0946,	best estimator catboost's best error=0.0946
INFO:flaml.automl: at 271.9s,	estimator catboost's best error=0.0946,	best estimator catboost's best error=0.0946
[flaml.automl: 09-25 04:51:01] {3174} INFO - iteration 1, current learner catboost
INFO:flaml.aut

In [32]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

Best ML leaner: catboost
Best hyperparmeter config: {'early_stopping_rounds': 11, 'learning_rate': 0.1604199347807429, 'n_estimators': 532}
Training duration of best run: 310.8 s


In [33]:
automl.model.estimator

In [34]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

y_pred = automl.predict(X_test)

print('r2', '=', r2_score(y_test, y_pred))
print('mse', '=', mean_squared_error(y_test, y_pred))
print('rmse', '=', mean_squared_error(y_test, y_pred, squared=False))

r2 = 0.818740499150592
mse = 4943.266999173011
rmse = 70.3083707617593


In [35]:
performance_mdl_reg = pd.DataFrame(columns = ["Tema", "Área", "Algoritmo", "R2", "MSE", "RMSE"])
performance_mdl_reg = performance_mdl_reg.append({"Tema": "Benefícios", "Área": "Residual - BPC - Valor Per Capta", "Algoritmo": "CatBoost",
                                          "R2": r2_score(y_test, y_pred),
                                          "MSE": mean_squared_error(y_test, y_pred),
                                          "RMSE": mean_squared_error(y_test, y_pred, squared=False)}, ignore_index = True)

performance_mdl_reg.to_sql('performance_mdl_reg', con=conn, if_exists='append', index=False)

In [36]:
import pickle

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_cb_beneficios_resid_bpc_valor_per_capta.pkl', 'wb') as f:
    pickle.dump(automl, f, pickle.HIGHEST_PROTOCOL)

##### Ensemble

In [49]:
from sklearn.linear_model import LinearRegression
from flaml.ml import sklearn_metric_loss_score
import pickle

pred_train_ensem = pd.DataFrame()
pred_test_ensem = pd.DataFrame()

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_xgb_beneficios_resid_bpc_valor_per_capta.pkl', 'rb') as f:
    mdl_xgb = pickle.load(f)

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_lgbm_beneficios_resid_bpc_valor_per_capta.pkl', 'rb') as f:
    mdl_lgbm = pickle.load(f)

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_cb_beneficios_resid_bpc_valor_per_capta.pkl', 'rb') as f:
    mdl_cb = pickle.load(f)

pred_train_ensem["xgb_pred_resid_bpc_valor_per_capta"] = mdl_xgb.predict(X_train)
pred_test_ensem["xgb_pred_resid_bpc_valor_per_capta"] = mdl_xgb.predict(X_test)
pred_train_ensem["lgbm_pred_resid_bpc_valor_per_capta"] = mdl_lgbm.predict(X_train)
pred_test_ensem["lgbm_pred_resid_bpc_valor_per_capta"] = mdl_lgbm.predict(X_test)
pred_train_ensem["cb_pred_resid_bpc_valor_per_capta"] = mdl_cb.predict(X_train)
pred_test_ensem["cb_pred_resid_bpc_valor_per_capta"] = mdl_cb.predict(X_test)

pred_train_ensem["y_train"] = y_train.reset_index(drop=True)
pred_test_ensem["y_test"] = y_test.reset_index(drop=True)

X_train_ensem = pred_train_ensem.drop(["y_train"], 1)
X_test_ensem = pred_test_ensem.drop(["y_test"], 1)
y_train_ensem = pred_train_ensem["y_train"]
y_test_ensem = pred_test_ensem["y_test"]

clf = LinearRegression().fit(X_train_ensem, y_train_ensem)
y_pred = clf.predict(X_test_ensem)
print('r2', '=', r2_score(y_test_ensem, y_pred))
print('mse', '=', mean_squared_error(y_test_ensem, y_pred))
print('rmse', '=', mean_squared_error(y_test_ensem, y_pred, squared=False))

performance_mdl_reg = pd.DataFrame(columns = ["Tema", "Área", "Algoritmo", "R2", "MSE", "RMSE"])
performance_mdl_reg = performance_mdl_reg.append({"Tema": "Benefícios", "Área": "Residual - BPC - Valor Per Capta", "Algoritmo": "Ensemble",
                                          "R2": r2_score(y_test_ensem, y_pred),
                                          "MSE": mean_squared_error(y_test_ensem, y_pred),
                                          "RMSE": mean_squared_error(y_test_ensem, y_pred, squared=False)}, ignore_index = True)

performance_mdl_reg.to_sql('performance_mdl_reg', con=conn, if_exists='append', index=False)

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/beneficios_resid_ensemb_valor_per_capta.pkl', 'wb') as f:
    pickle.dump(clf, f, pickle.HIGHEST_PROTOCOL)

[05:26:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
r2 = 0.9382104347352489
mse = 1685.1106697036275
rmse = 41.050099509058775


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


In [50]:
preds_beneficios = pd.DataFrame()

X_preds = agg_municipios.drop(["codigo_ibge", "nome", "siafi_id"], 1)
X_preds["ano"] = 2022
preds_beneficios["xgb_pred_resid_bpc_valor_per_capta"] = mdl_xgb.predict(X_preds)
preds_beneficios["lgbm_pred_resid_bpc_valor_per_capta"] = mdl_lgbm.predict(X_preds)
preds_beneficios["cb_pred_resid_bpc_valor_per_capta"] = mdl_cb.predict(X_preds)

preds_beneficios["candido_pred_resid_bpc_valor_per_capta"] = clf.predict(preds_beneficios)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [52]:
preds_beneficios_db = pd.read_sql('select * from preds_beneficios', con = conn)
preds_beneficios_db = pd.concat([preds_beneficios_db, preds_beneficios], axis = 1)
preds_beneficios_db.to_sql('preds_beneficios', con=conn, if_exists='replace', index=False)

### Bolsa Família

#### Valor Per Capta

In [53]:
dados_agg = pd.DataFrame(columns = ["ano_mes", "siafi_id", "cont_parcelas",	"valor_parcela_agg"])
dados_agg = pd.concat([dados_agg, bolsa_familia])
dados_agg["siafi_id"] = dados_agg["siafi_id"].astype(float).astype(int).astype(str)
dados_agg["ano"] = dados_agg["ano_mes"].astype(str).str.slice(0, 4)
dados_agg = dados_agg[["siafi_id", "ano", "valor_parcela_agg"]].groupby(["ano", "siafi_id"]).sum().reset_index()

In [54]:
df_melt = agg_municipios[["siafi_id", "pop_2014", "pop_2015", "pop_2016", "pop_2017", "pop_2018", "pop_2019", "pop_2020", "pop_2021"]]
pop = pd.melt(df_melt, id_vars=["siafi_id"], value_vars=["pop_2014", "pop_2015", "pop_2016", "pop_2017", "pop_2018", "pop_2019", "pop_2020", "pop_2021"])
pop["variable"] = pop["variable"].replace("pop_", "", regex = True)
pop = pop.rename(columns = {"variable": "ano", "value": "pop"})
pop["siafi_id"] = pop["siafi_id"].astype(str)

In [55]:
dados_agg = pd.merge(dados_agg, pop, on = ["siafi_id", "ano"], how = "left")
dados_agg = dados_agg.dropna(subset = ["pop"])
dados_agg["pop"] = dados_agg["pop"].astype(int)
dados_agg["valor_parcela_per_capta"] = dados_agg["valor_parcela_agg"] / dados_agg["pop"]
dados_agg = dados_agg.round({'valor_parcela_per_capta': 2})

In [56]:
list_cols = agg_municipios.columns.tolist()
list_cols.append("ano")
list_cols.append("valor_parcela_per_capta")
dataset = pd.DataFrame(columns = list_cols)

dados_agg = dados_agg.reset_index(drop=True)
dados_agg = dados_agg[["ano", "siafi_id", "valor_parcela_per_capta"]]
agg_municipios["siafi_id"] = agg_municipios["siafi_id"].astype(str)

for ano in range(2014, 2022):
  ds_s_value = dados_agg[dados_agg["ano"]==str(ano)].reset_index(drop=True)
  ds_s_value = pd.merge(ds_s_value, agg_municipios, how = "left", on = "siafi_id") 

  ds_n_value = agg_municipios[~agg_municipios['siafi_id'].isin(ds_s_value["siafi_id"].unique().tolist())]
  ds_n_value["ano"] = str(ano)
  ds_n_value["valor_parcela_per_capta"] = 0
  ds_n_value = ds_n_value.reset_index(drop=True)

  dataset = pd.concat([dataset, pd.concat([ds_n_value, ds_s_value], ignore_index = True)], ignore_index = True)

dataset["ano"] = dataset["ano"].astype(int)

In [57]:
X_train = dataset[(dataset["ano"]!=2021)].drop(["codigo_ibge", "nome", "siafi_id", "valor_parcela_per_capta"], 1)
y_train = dataset[(dataset["ano"]!=2021)]["valor_parcela_per_capta"]
X_test = dataset[dataset["ano"]==2021].drop(["codigo_ibge", "nome", "siafi_id", "valor_parcela_per_capta"], 1)
y_test = dataset[dataset["ano"]==2021]["valor_parcela_per_capta"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


##### XGBoost

In [60]:
from flaml import AutoML

automl = AutoML()
settings = {
    "time_budget": 360,  
    "metric": 'r2', 
    "estimator_list": ['xgboost'],  
    "task": 'regression'    
}

automl.fit(X_train = X_train, y_train = y_train, **settings)

[flaml.automl: 09-25 06:20:58] {2600} INFO - task = regression
INFO:flaml.automl:task = regression
[flaml.automl: 09-25 06:20:58] {2602} INFO - Data split method: uniform
INFO:flaml.automl:Data split method: uniform
[flaml.automl: 09-25 06:20:58] {2605} INFO - Evaluation method: holdout
INFO:flaml.automl:Evaluation method: holdout
[flaml.automl: 09-25 06:20:58] {2727} INFO - Minimizing error metric: 1-r2
INFO:flaml.automl:Minimizing error metric: 1-r2
[flaml.automl: 09-25 06:20:58] {2869} INFO - List of ML learners in AutoML Run: ['xgboost']
INFO:flaml.automl:List of ML learners in AutoML Run: ['xgboost']
[flaml.automl: 09-25 06:20:58] {3174} INFO - iteration 0, current learner xgboost
INFO:flaml.automl:iteration 0, current learner xgboost
[flaml.automl: 09-25 06:21:01] {3308} INFO - Estimated sufficient time budget=23390s. Estimated necessary time budget=23s.
INFO:flaml.automl:Estimated sufficient time budget=23390s. Estimated necessary time budget=23s.
[flaml.automl: 09-25 06:21:01] 

In [61]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

Best ML leaner: xgboost
Best hyperparmeter config: {'n_estimators': 352, 'max_leaves': 489, 'min_child_weight': 6.392462455671886, 'learning_rate': 0.18085024148873766, 'subsample': 0.8413048297641477, 'colsample_bylevel': 0.49309776988003695, 'colsample_bytree': 0.5217724923789486, 'reg_alpha': 0.03939155236090648, 'reg_lambda': 0.6115970190579195}
Training duration of best run: 176.7 s


In [62]:
automl.model.estimator

XGBRegressor(colsample_bylevel=0.49309776988003695,
             colsample_bytree=0.5217724923789486, grow_policy='lossguide',
             learning_rate=0.18085024148873766, max_depth=0, max_leaves=489,
             min_child_weight=6.392462455671886, n_estimators=352, n_jobs=-1,
             reg_alpha=0.03939155236090648, reg_lambda=0.6115970190579195,
             subsample=0.8413048297641477, tree_method='hist',
             use_label_encoder=False, verbosity=0)

In [63]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

y_pred = automl.predict(X_test)

print('r2', '=', r2_score(y_test, y_pred))
print('mse', '=', mean_squared_error(y_test, y_pred))
print('rmse', '=', mean_squared_error(y_test, y_pred, squared=False))

r2 = 0.979245127941942
mse = 599.8122798217496
rmse = 24.491065305979436


In [64]:
performance_mdl_reg = pd.DataFrame(columns = ["Tema", "Área", "Algoritmo", "R2", "MSE", "RMSE"])
performance_mdl_reg = performance_mdl_reg.append({"Tema": "Benefícios", "Área": "Residual - Bolsa Família - Valor Per Capta", "Algoritmo": "XGBoost",
                                          "R2": r2_score(y_test, y_pred),
                                          "MSE": mean_squared_error(y_test, y_pred),
                                          "RMSE": mean_squared_error(y_test, y_pred, squared=False)}, ignore_index = True)

performance_mdl_reg.to_sql('performance_mdl_reg', con=conn, if_exists='append', index=False)

In [65]:
import pickle

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_xgb_beneficios_resid_bolsa_familia_valor_per_capta.pkl', 'wb') as f:
    pickle.dump(automl, f, pickle.HIGHEST_PROTOCOL)

##### LGBM

In [66]:
from flaml import AutoML

automl = AutoML()
settings = {
    "time_budget": 360,  
    "metric": 'r2', 
    "estimator_list": ['lgbm'],  
    "task": 'regression'    
}

automl.fit(X_train = X_train, y_train = y_train, **settings)

[flaml.automl: 09-25 06:29:05] {2600} INFO - task = regression
INFO:flaml.automl:task = regression
[flaml.automl: 09-25 06:29:05] {2602} INFO - Data split method: uniform
INFO:flaml.automl:Data split method: uniform
[flaml.automl: 09-25 06:29:05] {2605} INFO - Evaluation method: holdout
INFO:flaml.automl:Evaluation method: holdout
[flaml.automl: 09-25 06:29:05] {2727} INFO - Minimizing error metric: 1-r2
INFO:flaml.automl:Minimizing error metric: 1-r2
[flaml.automl: 09-25 06:29:05] {2869} INFO - List of ML learners in AutoML Run: ['lgbm']
INFO:flaml.automl:List of ML learners in AutoML Run: ['lgbm']
[flaml.automl: 09-25 06:29:05] {3174} INFO - iteration 0, current learner lgbm
INFO:flaml.automl:iteration 0, current learner lgbm
[flaml.automl: 09-25 06:29:07] {3308} INFO - Estimated sufficient time budget=18268s. Estimated necessary time budget=18s.
INFO:flaml.automl:Estimated sufficient time budget=18268s. Estimated necessary time budget=18s.
[flaml.automl: 09-25 06:29:07] {3360} INFO 

In [67]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

Best ML leaner: lgbm
Best hyperparmeter config: {'n_estimators': 186, 'num_leaves': 1099, 'min_child_samples': 2, 'learning_rate': 0.11621588299658571, 'log_max_bin': 9, 'colsample_bytree': 0.4676132458366714, 'reg_alpha': 0.002468665539885623, 'reg_lambda': 0.0028201449346997825}
Training duration of best run: 106.3 s


In [68]:
automl.model.estimator

LGBMRegressor(colsample_bytree=0.4676132458366714,
              learning_rate=0.11621588299658571, max_bin=511,
              min_child_samples=2, n_estimators=186, num_leaves=1099,
              reg_alpha=0.002468665539885623, reg_lambda=0.0028201449346997825,
              verbose=-1)

In [69]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

y_pred = automl.predict(X_test)

print('r2', '=', r2_score(y_test, y_pred))
print('mse', '=', mean_squared_error(y_test, y_pred))
print('rmse', '=', mean_squared_error(y_test, y_pred, squared=False))

r2 = 0.9798467692000362
mse = 582.4249495774174
rmse = 24.133481919885025


In [70]:
performance_mdl_reg = pd.DataFrame(columns = ["Tema", "Área", "Algoritmo", "R2", "MSE", "RMSE"])
performance_mdl_reg = performance_mdl_reg.append({"Tema": "Benefícios", "Área": "Residual - Bolsa Família - Valor Per Capta", "Algoritmo": "LGBM",
                                          "R2": r2_score(y_test, y_pred),
                                          "MSE": mean_squared_error(y_test, y_pred),
                                          "RMSE": mean_squared_error(y_test, y_pred, squared=False)}, ignore_index = True)

performance_mdl_reg.to_sql('performance_mdl_reg', con=conn, if_exists='append', index=False)

In [71]:
import pickle

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_lgbm_beneficios_resid_bolsa_familia_valor_per_capta.pkl', 'wb') as f:
    pickle.dump(automl, f, pickle.HIGHEST_PROTOCOL)

##### CatBoost

In [72]:
from flaml import AutoML

automl = AutoML()
settings = {
    "time_budget": 360,  
    "metric": 'r2', 
    "estimator_list": ['catboost'],  
    "task": 'regression'    
}

automl.fit(X_train = X_train, y_train = y_train, **settings)

[flaml.automl: 09-25 06:35:43] {2600} INFO - task = regression
INFO:flaml.automl:task = regression
[flaml.automl: 09-25 06:35:43] {2602} INFO - Data split method: uniform
INFO:flaml.automl:Data split method: uniform
[flaml.automl: 09-25 06:35:43] {2605} INFO - Evaluation method: holdout
INFO:flaml.automl:Evaluation method: holdout
[flaml.automl: 09-25 06:35:43] {2727} INFO - Minimizing error metric: 1-r2
INFO:flaml.automl:Minimizing error metric: 1-r2
[flaml.automl: 09-25 06:35:43] {2869} INFO - List of ML learners in AutoML Run: ['catboost']
INFO:flaml.automl:List of ML learners in AutoML Run: ['catboost']
[flaml.automl: 09-25 06:35:43] {3174} INFO - iteration 0, current learner catboost
INFO:flaml.automl:iteration 0, current learner catboost
[flaml.automl: 09-25 06:41:39] {3308} INFO - Estimated sufficient time budget=3562808s. Estimated necessary time budget=3563s.
INFO:flaml.automl:Estimated sufficient time budget=3562808s. Estimated necessary time budget=3563s.
[flaml.automl: 09-2

In [73]:
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

Best ML leaner: catboost
Best hyperparmeter config: {'early_stopping_rounds': 10, 'learning_rate': 0.09999999999999996, 'n_estimators': 718}
Training duration of best run: 355.6 s


In [74]:
automl.model.estimator

In [75]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

y_pred = automl.predict(X_test)

print('r2', '=', r2_score(y_test, y_pred))
print('mse', '=', mean_squared_error(y_test, y_pred))
print('rmse', '=', mean_squared_error(y_test, y_pred, squared=False))

r2 = 0.9337642117973025
mse = 1914.2030370756154
rmse = 43.75160610852607


In [76]:
performance_mdl_reg = pd.DataFrame(columns = ["Tema", "Área", "Algoritmo", "R2", "MSE", "RMSE"])
performance_mdl_reg = performance_mdl_reg.append({"Tema": "Benefícios", "Área": "Residual - Bolsa Família - Valor Per Capta", "Algoritmo": "CatBoost",
                                          "R2": r2_score(y_test, y_pred),
                                          "MSE": mean_squared_error(y_test, y_pred),
                                          "RMSE": mean_squared_error(y_test, y_pred, squared=False)}, ignore_index = True)

performance_mdl_reg.to_sql('performance_mdl_reg', con=conn, if_exists='append', index=False)

In [77]:
import pickle

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_cb_beneficios_resid_bolsa_familia_valor_per_capta.pkl', 'wb') as f:
    pickle.dump(automl, f, pickle.HIGHEST_PROTOCOL)

##### Ensemble

In [78]:
from sklearn.linear_model import LinearRegression
from flaml.ml import sklearn_metric_loss_score
import pickle

pred_train_ensem = pd.DataFrame()
pred_test_ensem = pd.DataFrame()

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_xgb_beneficios_resid_bolsa_familia_valor_per_capta.pkl', 'rb') as f:
    mdl_xgb = pickle.load(f)

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_lgbm_beneficios_resid_bolsa_familia_valor_per_capta.pkl', 'rb') as f:
    mdl_lgbm = pickle.load(f)

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/mdl_cb_beneficios_resid_bolsa_familia_valor_per_capta.pkl', 'rb') as f:
    mdl_cb = pickle.load(f)

pred_train_ensem["xgb_pred_resid_bolsa_familia_valor_per_capta"] = mdl_xgb.predict(X_train)
pred_test_ensem["xgb_pred_resid_bolsa_familia_valor_per_capta"] = mdl_xgb.predict(X_test)
pred_train_ensem["lgbm_pred_resid_bolsa_familia_valor_per_capta"] = mdl_lgbm.predict(X_train)
pred_test_ensem["lgbm_pred_resid_bolsa_familia_valor_per_capta"] = mdl_lgbm.predict(X_test)
pred_train_ensem["cb_pred_resid_bolsa_familia_valor_per_capta"] = mdl_cb.predict(X_train)
pred_test_ensem["cb_pred_resid_bolsa_familia_valor_per_capta"] = mdl_cb.predict(X_test)

pred_train_ensem["y_train"] = y_train.reset_index(drop=True)
pred_test_ensem["y_test"] = y_test.reset_index(drop=True)

X_train_ensem = pred_train_ensem.drop(["y_train"], 1)
X_test_ensem = pred_test_ensem.drop(["y_test"], 1)
y_train_ensem = pred_train_ensem["y_train"]
y_test_ensem = pred_test_ensem["y_test"]

clf = LinearRegression().fit(X_train_ensem, y_train_ensem)
y_pred = clf.predict(X_test_ensem)
print('r2', '=', r2_score(y_test_ensem, y_pred))
print('mse', '=', mean_squared_error(y_test_ensem, y_pred))
print('rmse', '=', mean_squared_error(y_test_ensem, y_pred, squared=False))

performance_mdl_reg = pd.DataFrame(columns = ["Tema", "Área", "Algoritmo", "R2", "MSE", "RMSE"])
performance_mdl_reg = performance_mdl_reg.append({"Tema": "Benefícios", "Área": "Residual - Bolsa Família - Valor Per Capta", "Algoritmo": "Ensemble",
                                          "R2": r2_score(y_test_ensem, y_pred),
                                          "MSE": mean_squared_error(y_test_ensem, y_pred),
                                          "RMSE": mean_squared_error(y_test_ensem, y_pred, squared=False)}, ignore_index = True)

performance_mdl_reg.to_sql('performance_mdl_reg', con=conn, if_exists='append', index=False)

with open('/content/drive/MyDrive/geotesouro/modelagem/models_save/beneficios/beneficios_resid_bolsa_familia_ensemb_valor_per_capta.pkl', 'wb') as f:
    pickle.dump(clf, f, pickle.HIGHEST_PROTOCOL)

r2 = 0.979329629660269
mse = 597.3701945042969
rmse = 24.441157797950098


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


In [79]:
preds_beneficios = pd.DataFrame()

X_preds = agg_municipios.drop(["codigo_ibge", "nome", "siafi_id"], 1)
X_preds["ano"] = 2022
preds_beneficios["xgb_pred_resid_bolsa_familia_valor_per_capta"] = mdl_xgb.predict(X_preds)
preds_beneficios["lgbm_pred_resid_bolsa_familia_valor_per_capta"] = mdl_lgbm.predict(X_preds)
preds_beneficios["cb_pred_resid_bolsa_familia_valor_per_capta"] = mdl_cb.predict(X_preds)

preds_beneficios["candido_pred_resid_bolsa_familia_valor_per_capta"] = clf.predict(preds_beneficios)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [80]:
preds_beneficios

,xgb_pred_resid_bolsa_familia_valor_per_capta,lgbm_pred_resid_bolsa_familia_valor_per_capta,cb_pred_resid_bolsa_familia_valor_per_capta,candido_pred_resid_bolsa_familia_valor_per_capta
0,217.498886,216.332930,193.240600,217.452501
1,61.152328,62.376521,103.305557,60.926460
2,63.042645,63.462703,93.464984,62.679473
3,45.085751,45.297559,82.844477,44.477256
4,525.508850,523.392524,414.548172,526.679446
...,...,...,...,...
5565,630.448730,630.283883,515.962440,632.571518
5566,390.774536,388.605576,399.048211,389.468110
5567,46.123177,45.401411,64.464920,45.478590
5568,265.010651,264.646972,335.398956,263.319754


In [81]:
preds_beneficios_db = pd.read_sql('select * from preds_beneficios', con = conn)
preds_beneficios_db = pd.concat([preds_beneficios_db, preds_beneficios], axis = 1)
preds_beneficios_db.to_sql('preds_beneficios', con=conn, if_exists='replace', index=False)